In [ ]:
from attention_project_utils import * 
import glob 
import numpy as np
import pandas as pd
from scipy import spatial,stats
from statistics import NormalDist
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import matplotlib.colors as colors
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.lines as mlines
import seaborn as sns
import tifffile

In [ ]:
def reject_outliers(data, m = 3.5):
    d = np.abs(data - np.median(data))
    mdev = np.median(d)
    s = d/mdev if mdev else np.zero(len(d))
    return data[s<m]

In [ ]:
main_dir_list = ["F:\\LCr_UV_HSA_Lum_Low_1\\20230312"]*5+["F:\\LCr_UV_HSA_Lum_Low_2\\20230409"]*3+["F:\\LCr_UV_HSA_Lum_Low_2\\20230410"]*2
fish_index_list = ['F1_10_40','F2_12_20','F4_16_00','F5_18_00','F6_20_00','F2_13_00','F4_17_00','F5_19_00','F2_13_00','F4_16_00']

visuomotor_list = ['ns','Prey','Freezing','Luminance','motor_Freezing','motor_PC']

time_len = 39
plane_no = 9

output_dir = 'C:\\Users\\yxton\\Documents\\Code\\2p\\freezing_project\\clustering_files_low_elevation'
create_directory(output_dir)


df_reg_clusters = pd.read_csv(output_dir+"\\low_elev_registered_10fish_clustering_withfish_withROIindex.csv")
df_VMV_all = pd.read_csv(output_dir+'\\df_VMV_10fish_v5_withcluster.csv')
df_trace_all = pd.read_csv(output_dir+'\\trace_10fish_v6.csv')



unnamed_column = 0
random_no = 2000

npy_trace_all = np.array(df_trace_all)
npy_trace_all.shape

In [ ]:
df_trace_all_v5 = pd.read_csv(output_dir+'\\trace_10fish_v5.csv')

In [ ]:
df_reg_clusters['cluster'] = df_VMV_all.loc[df_reg_clusters.label,'cluster']
df_reg_clusters

In [ ]:
clusters_large = []
df_id_list = []
trace_filtered_list = []

df_filtered_reg_clusters = pd.DataFrame(columns = ['index','x','y','z','cluster','date','fish'])


clusters = np.unique(df_reg_clusters.cluster)
for cluster_i in clusters:
    temp_df = df_reg_clusters[df_reg_clusters['cluster']== cluster_i]
    if temp_df.shape[0]>10:
        clusters_large.append(cluster_i)
        
for main_dir,fish_index in zip(main_dir_list,fish_index_list):
    temp_date = main_dir.split('\\')[-1]
    temp_fish = fish_index[:2]
    df_id = str(temp_date)+str(temp_fish)
    df_id_list.append(df_id)
    locals()['df_'+df_id] = df_reg_clusters[np.logical_and(df_reg_clusters['date'] == int(temp_date),df_reg_clusters['fish'] == temp_fish)]
    for cluster_i in clusters_large:
        locals()['df_'+df_id+'_cluster'+str(cluster_i)] = locals()['df_'+df_id][locals()['df_'+df_id]['cluster']==cluster_i]
        
# loop for each neuron in df_reg_clusters
for i in range(0,df_reg_clusters.shape[0]):
    # first see whether it's a neuron in a cluster size>10:
    temp_cluster_id = df_reg_clusters['cluster'][i]
    if clusters_large.count(temp_cluster_id)>0:
        print(i)
        #basic info of this neuron
        temp_coor = list(df_reg_clusters.iloc[i,0+unnamed_column:3+unnamed_column]) #need to check whether there is an unnamed column
        temp_date = df_reg_clusters['date'][i]
        temp_fish = df_reg_clusters['fish'][i]
        temp_df_id = str(temp_date)+str(temp_fish)
        
        dr_list = []
        #calculate dr of this neuron
        for df_id in df_id_list:
            if df_id != temp_df_id and locals()['df_'+df_id+'_cluster'+str(temp_cluster_id)].shape[0]>0:
                temp_fish_coor_array = np.array(locals()['df_'+df_id+'_cluster'+str(temp_cluster_id)].iloc[:,0+unnamed_column:3+unnamed_column])
                distance,index = spatial.KDTree(temp_fish_coor_array).query(temp_coor)
                dr_list.append(distance)
            if df_id == temp_df_id and locals()['df_'+df_id+'_cluster'+str(temp_cluster_id)].shape[0]>1:
                temp_fish_coor_array = np.array(locals()['df_'+df_id+'_cluster'+str(temp_cluster_id)].iloc[:,0+unnamed_column:3+unnamed_column])
                distance,index = spatial.KDTree(temp_fish_coor_array).query(temp_coor,k=2)
                dr_list.append(distance[1])
        mean_dr = np.mean(reject_outliers(np.array(dr_list)))
                                                                                    
        #calculate and fit dn normal distribution of this neuron for 1000 times
        
        random_sampling = np.random.choice(df_reg_clusters.shape[0],random_no, replace=True)
        temp_fish_coor_array_random = np.array(df_reg_clusters.iloc[random_sampling,0+unnamed_column:3+unnamed_column])
        dn = spatial.distance.cdist(np.array([temp_coor]*random_no),temp_fish_coor_array_random)
        mu,s = stats.norm.fit(dn)

        # reject the null hypothesis of random sampling if a neuron has a spatial P value < 0.025.
        if NormalDist(mu, s).cdf(mean_dr)<0.025:
            df_ii = df_filtered_reg_clusters.shape[0]
            df_filtered_reg_clusters.loc[df_ii,:] = [i,temp_coor[0],temp_coor[1],temp_coor[2],temp_cluster_id,temp_date,temp_fish]
            trace_filtered_list.append(npy_trace_all[i,:])

In [ ]:
+
df_filtered_reg_clusters.to_csv(output_dir+'\\df_filtered_reg_clusters.csv',index=False)

In [ ]:
df_filtered_reg_clusters = pd.read_csv(output_dir+'\\df_filtered_reg_clusters.csv')

In [ ]:
df_VMV_filtered = df_VMV_all.loc[df_filtered_reg_clusters['index'],:]
npy_VMV_filtered = df_VMV_filtered.iloc[:,:7].to_numpy(dtype = 'float64')

# df_trace_filtered = df_trace_all.loc[df_filtered_reg_clusters['index'],:]
# npy_trace_filtered = df_trace_filtered.to_numpy(dtype = 'float64')

clusters = np.array(df_filtered_reg_clusters['cluster'],dtype = 'float64')
npy_VMV_filtered_cluster = np.c_[npy_VMV_filtered,clusters]



In [ ]:
df_VMV_filtered

In [ ]:
np.unique(clusters).shape

In [ ]:
np_trace_filtered = npy_trace_all[df_filtered_reg_clusters['index'].to_list(),:]

In [ ]:
# vmv of different clusters

fig,[ax2,ax1,ax3]= plt.subplots(1,3,figsize = (19,15), gridspec_kw=dict(width_ratios=[.07, .85, .08]),dpi=800)

# fig = plt.figure(figsize = (12,14))
# ax2 = plt.subplot(131, gridspec_kw=dict(width_ratios=[.07, .85, .08]))
# ax1 = plt.subplot(132)
# ax3 = plt.subplot(235)
# d= sch.dendrogram(L, orientation='left', ax=ax4, color_threshold=threshold_ratio*d.max())

argsort = np.argsort(clusters)[::-1]
sns.heatmap(npy_VMV_filtered_cluster[argsort,:6],cmap = 'plasma',ax = ax1,vmin = -0.3,vmax = 0.7,cbar_ax = ax3)
sns.heatmap(npy_VMV_filtered_cluster[argsort,-1:],cmap = 'gist_ncar',cbar = False,ax = ax2)

ax1.set_yticks([]) 
ax1.set_xticklabels(['No Stim', '4°', '15°', 'Luminance', 'Freezing (motor)', 'Prey (motor)'],fontsize = 19)
for i in range(0,4):
    ax1.get_xticklabels()[i].set_color("maroon")
for i in range(4,6):
    ax1.get_xticklabels()[i].set_color("navy")
    
# ax2.set_ylabel('Neurons',fontsize = 18) 
ax2.set_yticks([]) 
ax2.set_xticklabels(['Cluster ID'],fontsize = 22)


line_x = np.where(clusters[argsort][:-1]-clusters[argsort][1:] !=0)
ax1.hlines(line_x , *ax1.get_xlim(),color = 'white',linewidth = 1.5)
ax2.hlines(line_x , *ax1.get_xlim(),color = 'white',linewidth = 1.5)


fig.subplots_adjust(wspace=0.02)

ax1.set_xlabel('Visuomotor Vectors',fontsize = 24)

ax3.set_ylabel('Averaged Visomotor Vector Score',fontsize = 24)
ax3.tick_params(labelsize=22)



# ax4.axis('off')

plt.show()

In [ ]:
# 2d scatter plot of distribution of each cluster
fig,ax1 = plt.subplots(1,1,figsize = (4,3),dpi=800)
cluster_idx=np.unique(clusters)
startcolor = 'crimson'
midcolor = '#FDFD96'
endcolor = 'dodgerblue'
cm = LinearSegmentedColormap.from_list( 'own2', [startcolor, midcolor, endcolor] )
scatter_info = np.zeros((len(cluster_idx),4))

color_cluster = np.zeros(npy_VMV_filtered_cluster.shape[0])

for cluster_i in range(0,len(cluster_idx)):
    i = int(cluster_idx[cluster_i])
    locals()['cluster_'+str(i)+'_roi_idx'] = list(np.where(clusters == i)[0])
    average_temp_cluster = np.average(npy_VMV_filtered_cluster[locals()['cluster_'+str(i)+'_roi_idx'],:6],axis = 0)
    scatter_info[cluster_i,0] = average_temp_cluster[4]
    scatter_info[cluster_i,1] = average_temp_cluster[5]
    scatter_info[cluster_i,2] = (average_temp_cluster[2]-average_temp_cluster[1])/(average_temp_cluster[2]+average_temp_cluster[1])
    color_cluster[locals()['cluster_'+str(i)+'_roi_idx']] = scatter_info[cluster_i,2]
    scatter_info[cluster_i,3] = len(locals()['cluster_'+str(i)+'_roi_idx'])
#     if scatter_info[cluster_i,3]>80:
#         ax1.annotate(str(i),(average_temp_cluster[4]-0.0055,average_temp_cluster[5]-0.005),fontsize = 5)
        
#     if scatter_info[cluster_i,0]>0.08 and scatter_info[cluster_i,1]>0.08:
#         index_temp = df_VMV_filtered.iloc[locals()['cluster_'+str(i)+'_roi_idx'],:].index
#         df_VMV_filtered.loc[index_temp,'cluster_type'] = 'both'
#     elif scatter_info[cluster_i,0]<scatter_info[cluster_i,1]:
#         index_temp = df_VMV_filtered.iloc[locals()['cluster_'+str(i)+'_roi_idx'],:].index
#         df_VMV_filtered.loc[index_temp,'cluster_type'] = 'PC'
#     else:
#         index_temp = df_VMV_filtered.iloc[locals()['cluster_'+str(i)+'_roi_idx'],:].index
#         df_VMV_filtered.loc[index_temp,'cluster_type'] = 'Freezing'
scatter_info[:,3] = scatter_info[:,3]/np.sum(scatter_info[:,3])
for cluster_i in range(0,len(cluster_idx)):
    i = int(cluster_idx[cluster_i])
    average_temp_cluster = np.average(npy_VMV_filtered_cluster[locals()['cluster_'+str(i)+'_roi_idx'],:6],axis = 0)
    if scatter_info[cluster_i,3]>0.04:
        ax1.annotate(str(i),(average_temp_cluster[4]-0.0095,average_temp_cluster[5]-0.007),fontsize = 8)

sc = plt.scatter(scatter_info[:,0],scatter_info[:,1],c = scatter_info[:,2],s = 1200*scatter_info[:,3],cmap = cm,vmin = -1,vmax = 1)
sc_cbar = plt.colorbar(sc,shrink=1,extend = 'both')
sc_cbar.set_label(label='Size Preference',size=10,labelpad = -10)
sc_cbar.set_ticks([-1,-0.5,0,0.5,1])
sc_cbar.set_ticklabels(['-1(4°)','-0.5','0','0.5','1(15°)'])
# sc_cbar,'Prefer 4°,'Prefer 15°'
plt.xlabel('Freezing (motor) score',fontsize = 10)
plt.ylabel('Prey (motor) score',fontsize = 10)
plt.xlim(-0.06,0.29)
plt.ylim(-0.06,0.29)
plt.xticks([0,0.1,0.2])
plt.yticks([0,0.1,0.2])
ax1.axhline(y = 0.08,xmin = 2/3.5,xmax = 0.95,color = 'dodgerblue',linewidth =2)
ax1.axhline(y = -0.05,xmin = 2/3.5,xmax = 0.95,color ='dodgerblue',linewidth =2)
ax1.axvline(x = 0.14,ymin = 0.1/3.5,ymax = 1.4/3.5,color = 'dodgerblue',linewidth =2)
ax1.axvline(x = 0.29-0.05*0.35,ymin = 0.1/3.5,ymax = 1.4/3.5,color = 'dodgerblue',linewidth =2)

ax1.axvline(x = 0.08,ymin = 2/3.5,ymax = 0.95,color = 'crimson',linewidth =2)
ax1.axvline(x = -0.05,ymin = 2/3.5,ymax = 0.95,color = 'crimson',linewidth =2)
ax1.axhline(y = 0.14,xmin = 0.1/3.5,xmax = 1.4/3.5,color = 'crimson',linewidth =2)
ax1.axhline(y = 0.29-0.05*0.35,xmin = 0.1/3.5,xmax = 1.4/3.5,color = 'crimson',linewidth =2)
# plt.plot([0.08,0.08],[0.08,1],linestyle='dashed',color = 'k')
# plt.plot([0.08,1],[0.08,0.08],linestyle='dashed',color = 'k')
# plt.title('Cluster Distribution(HSA neurons)')
plt.show()

In [ ]:
# vmv of different clusters

fig,[ax2,ax1,ax3]= plt.subplots(1,3,figsize = (12,15), gridspec_kw=dict(width_ratios=[.07, .89, .04]),dpi=800)

# fig = plt.figure(figsize = (12,14))
# ax2 = plt.subplot(131, gridspec_kw=dict(width_ratios=[.07, .85, .08]))
# ax1 = plt.subplot(132)
# ax3 = plt.subplot(235)
# d= sch.dendrogram(L, orientation='left', ax=ax4, color_threshold=threshold_ratio*d.max())
argsort = np.argsort(color_cluster)[::-1]
reshape_color_cluster = color_cluster.reshape((color_cluster.shape[0],1))

sns.heatmap(npy_VMV_filtered_cluster[argsort,:6],cmap = 'plasma',ax = ax1,vmin = -0.3,vmax = 0.7,cbar_ax = ax3)
sns.heatmap(reshape_color_cluster[argsort,:],cmap = cm,vmin = -1,vmax = 1,cbar = False,ax = ax2)

ax1.set_yticks([]) 
ax1.set_xticks([0.5,1.5,2.5,3.5,4.33,5.45])
ax1.set_xticklabels(['No Stim', '4°', '15°', 'Luminance', 'Freezing\n(motor)', 'Prey\n(motor)'],rotation = 20,fontsize = 24,ha = 'center')
for i in range(0,4):
    ax1.get_xticklabels()[i].set_color("maroon")
for i in range(4,6):
    ax1.get_xticklabels()[i].set_color("navy")
    
# ax2.set_ylabel('Neurons',fontsize = 18) 
ax2.set_yticks([]) 
ax2.set_xticklabels(['Cluster ID'],rotation = 20,fontsize = 24,ha = 'center')


line_x = np.where(reshape_color_cluster[argsort][:-1]-reshape_color_cluster[argsort][1:] !=0)
ax1.hlines(line_x , *ax1.get_xlim(),color = 'white',linewidth = 1.5)
ax2.hlines(line_x , *ax1.get_xlim(),color = 'white',linewidth = 1.5)


fig.subplots_adjust(wspace=0.02)

ax1.set_xlabel('Visuomotor Vectors',fontsize = 24)

ax3.set_ylabel('Average Visomotor Vector Score',fontsize = 24)
ax3.tick_params(labelsize=22)



# ax4.axis('off')

plt.show()

In [ ]:
#sort clusters to their freezing group, prey group, or both group, based on their VMV
#the threshold is 0.08 which is arbitrary
df_VMV_filtered

In [ ]:
df_active_neuron_normed_ratio = pd.DataFrame()
for main_dir,fish in zip(main_dir_list,fish_index_list):
    print(fish)
    date_no = main_dir.split('\\')[-1]
    vsinfo = pd.read_excel(main_dir+'\\behavior\\'+fish+'//vsinfo_'+date_no+'_'+fish+'.xlsx')
    if main_dir == "F:\\LCr_UV_HSA_Lum_Low_1\\20230312":
        bradyinfo_dir = glob.glob(main_dir+'\\bradyinfo\\Bradyinfo*'+fish+'.xlsx')[0]
    else:
        bradyinfo_dir = glob.glob(main_dir+'\\behavior\\'+fish+'\\Bradyinfo*'+fish+'.xlsx')[0]
                                  
    Brady_info = pd.read_excel(bradyinfo_dir,sheet_name = 'Bradyinfo')
    trial_no = vsinfo.shape[0]
    
    Index_nm_Freezing = []
    Index_nm_PC = []
    Index_nb_Freezing = []
    Index_nb_PC = []
    Index_Freezing = []
    Index_Prey = []
    
    for t in range(0,trial_no):
#         if Bad_Imaging_Quality_list.count(t) == 0:
        if vsinfo.Left_Stimulus_Type[t] != 'n' or vsinfo.Right_Stimulus_Type[t] != 'n':
            if Brady_info['Behavior_Classification'][t] == 'Freezing':
                Index_nm_Freezing.append(t)
            if Brady_info['Bradycardia_Classification'][t] == 0 and Brady_info['PC_trial'][t] == True:
                Index_nm_PC.append(t)
            if vsinfo.Left_Stimulus_Type[t] == 'm' or vsinfo.Right_Stimulus_Type[t] == 'm':
                if vsinfo.Left_Stimulus_Size[t] + vsinfo.Right_Stimulus_Size[t] == 20:
                    Index_Freezing.append(t)
                    if Brady_info['Behavior_Classification'][t] == 'Freezing':
                        Index_nb_Freezing.append(t)
                if vsinfo.Left_Stimulus_Size[t] + vsinfo.Right_Stimulus_Size[t] == 4:
                    Index_Prey.append(t)
                    if Brady_info['Bradycardia_Classification'][t] == 0 and Brady_info['PC_trial'][t] == True:
                        Index_nb_PC.append(t)
    RSM_freezing = 1/(len(Index_nb_Freezing)/len(Index_nm_Freezing))
    RMM_freezing = len(Index_nb_Freezing)/8
    print(RSM_freezing)
    print(RMM_freezing)
    
    RSM_Prey = 1/(len(Index_nb_PC)/len(Index_nm_PC))
    RMM_Prey = len(Index_nb_PC)/8
    print(RSM_Prey)
    print(RMM_Prey)
    
    temp_df_fish = df_VMV_filtered[np.logical_and(df_VMV_filtered['Fish']==fish,df_VMV_filtered['Date']==int(date_no))]
    for plane_idx in range(1,10):
        temp_df_plane = temp_df_fish[temp_df_fish['Plane']==plane_idx]

        if temp_df_plane.shape[0]>0:
            for t in range(0,trial_no):
                temp_dir = main_dir+'\\processed\\'+fish+'\\processed_suite2p\\plane'+str(plane_idx)+'\\DeltaF_F\\'+fish+'_Trial'+str(t+1)+".csv"
                locals()['DeltaF_F_'+str(t)] = pd.read_csv(temp_dir)
            
            rs_Freezing = np.zeros(temp_df_plane.shape[0])
            for trial in Index_Freezing:
                all_ROI_id = locals()['DeltaF_F_'+str(trial)].ROI_id
                sorter = np.argsort(all_ROI_id)
                idx_ROI = sorter[np.searchsorted(all_ROI_id, temp_df_plane.ROI_id, sorter=sorter)].values
                
                delta_f = locals()['DeltaF_F_'+str(trial)].iloc[idx_ROI,:39].to_numpy()
                sti_on_delta_f = np.average(delta_f[:,27:36],axis=1)

                rs_Freezing = rs_Freezing +(sti_on_delta_f>1).astype(int)
            
            rm_Freezing = np.zeros(temp_df_plane.shape[0])
            for trial in Index_nm_Freezing:
                all_ROI_id = locals()['DeltaF_F_'+str(trial)].ROI_id
                sorter = np.argsort(all_ROI_id)
                idx_ROI = sorter[np.searchsorted(all_ROI_id, temp_df_plane.ROI_id, sorter=sorter)].values
                
                delta_f = locals()['DeltaF_F_'+str(trial)].iloc[idx_ROI,:39].to_numpy()
                sti_on_delta_f = np.average(delta_f[:,27:36],axis=1)
                rm_Freezing = rm_Freezing +(sti_on_delta_f>1).astype(int)
                
            rs_PC = np.zeros(temp_df_plane.shape[0])
            for trial in Index_Prey:
                all_ROI_id = locals()['DeltaF_F_'+str(trial)].ROI_id
                sorter = np.argsort(all_ROI_id)
                idx_ROI = sorter[np.searchsorted(all_ROI_id, temp_df_plane.ROI_id, sorter=sorter)].values
                
                delta_f = locals()['DeltaF_F_'+str(trial)].iloc[idx_ROI,:39].to_numpy()
                sti_on_delta_f = np.average(delta_f[:,27:36],axis=1)
                rs_PC = rs_PC +(sti_on_delta_f>1).astype(int)
            
            rm_PC = np.zeros(temp_df_plane.shape[0])
            for trial in Index_nm_PC:
                all_ROI_id = locals()['DeltaF_F_'+str(trial)].ROI_id
                sorter = np.argsort(all_ROI_id)
                idx_ROI = sorter[np.searchsorted(all_ROI_id, temp_df_plane.ROI_id, sorter=sorter)].values
                
                delta_f = locals()['DeltaF_F_'+str(trial)].iloc[idx_ROI,:39].to_numpy()
                sti_on_delta_f = np.average(delta_f[:,27:36],axis=1)
                rm_PC = rm_PC +(sti_on_delta_f>1).astype(int)
            
            ratio_Freezing = (rs_Freezing/8)/(rm_Freezing/len(Index_nm_Freezing))
            ratio_Freezing[np.where(np.isinf(ratio_Freezing))[0]] = RMM_freezing # pure sensory, after normalization it would be 0
            ratio_Freezing[np.where(np.isnan(ratio_Freezing))[0]] = RSM_freezing/2+RMM_freezing/2
#             ratio_Freezing[np.where(ratio_Freezing==0)[0]] = RSM_freezing
            
            ratio_PC = (rs_PC/8)/(rm_PC/len(Index_nm_PC))
            ratio_PC[np.where(np.isinf(ratio_PC))[0]] = RMM_Prey
            ratio_PC[np.where(np.isnan(ratio_PC))[0]] = RSM_Prey/2+RMM_Prey/2
#             ratio_PC[np.where(ratio_PC==0)[0]] = RSM_Prey

            
            temp_df_plane['normed_ratio_Freezing'] = (ratio_Freezing-RSM_freezing)/(RMM_freezing-RSM_freezing)
            temp_df_plane['normed_ratio_PC'] = (ratio_PC-RSM_Prey)/(RMM_Prey-RSM_Prey)
            df_active_neuron_normed_ratio = df_active_neuron_normed_ratio.append(temp_df_plane)


In [ ]:
#Sensory-Motor Index 
#first select cluster
#then show their sensory motor index on

df_active_neuron_normed_ratio

In [ ]:
df_active_neuron_normed_ratio.to_csv(output_dir+'\\df_normed_ratio_hsa.csv',index=False)

In [ ]:
df_active_neuron_normed_ratio = pd.read_csv('C:\\Users\\yxton\\Documents\\Code\\2p\\freezing_project\\clustering_files_low_elevation\\df_normed_ratio_hsa.csv')

In [ ]:
df_active_neuron_normed_ratio_Freezing = df_active_neuron_normed_ratio[df_active_neuron_normed_ratio['cluster_type']!='PC']
df_active_neuron_normed_ratio_PC = df_active_neuron_normed_ratio[df_active_neuron_normed_ratio['cluster_type']!='Freezing']

cluster_idx_Freezing = np.unique(df_active_neuron_normed_ratio_Freezing.cluster)
cluster_idx_PC = np.unique(df_active_neuron_normed_ratio_PC.cluster)



In [ ]:
# freezing 
fig,ax1 = plt.subplots(1,1,figsize = (4,1),dpi=800)
scatter_info = np.zeros((len(cluster_idx_Freezing),4))
temp_df = df_active_neuron_normed_ratio_Freezing
temp_np = df_active_neuron_normed_ratio_Freezing.iloc[:,:6].to_numpy()
for cluster_i in range(0,len(cluster_idx_Freezing)):
    i = int(cluster_idx_Freezing[cluster_i])
    locals()['cluster_'+str(i)+'_roi_idx'] = list(np.where(temp_df.cluster == i)[0])
    scatter_info[cluster_i,0] = np.average(temp_df.iloc[locals()['cluster_'+str(i)+'_roi_idx'],-2])
    average_temp_cluster = np.average(temp_np[locals()['cluster_'+str(i)+'_roi_idx'],:4],axis = 0)
    scatter_info[cluster_i,1] = (average_temp_cluster[2]-average_temp_cluster[1])/(average_temp_cluster[2]+average_temp_cluster[1])
    scatter_info[cluster_i,2] = int(len(locals()['cluster_'+str(i)+'_roi_idx']))
    scatter_info[cluster_i,3] = int(i)
    if i==100:
        ax1.annotate(str(i),(scatter_info[cluster_i,0]-0.025,0.015),fontsize = 4)
    elif scatter_info[cluster_i,2]>300:
        ax1.annotate(str(i),(scatter_info[cluster_i,0]-0.015,0),fontsize = 4)
sc = plt.scatter(scatter_info[:,0],np.zeros(scatter_info.shape[0]),c = scatter_info[:,1],s = scatter_info[:,2]/5,cmap = cm,vmin = -1,vmax = 1,alpha = 0.6)
sc_cbar = plt.colorbar(sc,shrink=1,extend = 'both')
sc_cbar.set_label(label='Size Preference',size=12)
sc_cbar.set_ticks([-1,-0.5,0,0.5,1])
sc_cbar.set_ticklabels(['-1','-0.5','0','0.5','1'])
plt.xlabel('Score of Freezing(motor)',fontsize = 12)
# plt.xlim(-0.05,1.05)
plt.xticks([0,0.5,1.0])
plt.title('Cluster Distribution in HSA dataset(Freezing)')
plt.show()

fig,ax1 = plt.subplots(1,1,figsize = (4,1),dpi=800)
scatter_info = np.zeros((len(cluster_idx_PC),4))
temp_df = df_active_neuron_normed_ratio_PC
temp_np = df_active_neuron_normed_ratio_PC.iloc[:,:6].to_numpy()
for cluster_i in range(0,len(cluster_idx_PC)):
    i = int(cluster_idx_PC[cluster_i])
    locals()['cluster_'+str(i)+'_roi_idx'] = list(np.where(temp_df.cluster == i)[0])
    scatter_info[cluster_i,0] = np.average(temp_df.iloc[locals()['cluster_'+str(i)+'_roi_idx'],-1])
    average_temp_cluster = np.average(temp_np[locals()['cluster_'+str(i)+'_roi_idx'],:4],axis = 0)
    scatter_info[cluster_i,1] = (average_temp_cluster[2]-average_temp_cluster[1])/(average_temp_cluster[2]+average_temp_cluster[1])
    scatter_info[cluster_i,2] = int(len(locals()['cluster_'+str(i)+'_roi_idx']))
    scatter_info[cluster_i,3] = int(i)
    if i==95:
        ax1.annotate(str(i),(scatter_info[cluster_i,0]-0.015,0.015),fontsize = 4)
    elif i==77:
        ax1.annotate(str(i),(scatter_info[cluster_i,0]-0.034,0),fontsize = 4)
    elif i==100:
        ax1.annotate(str(i),(scatter_info[cluster_i,0]-0.02,0),fontsize = 4)
    elif scatter_info[cluster_i,2]>300:
        ax1.annotate(str(i),(scatter_info[cluster_i,0]-0.015,0),fontsize = 4)
sc = plt.scatter(scatter_info[:,0],np.zeros(scatter_info.shape[0]),c = scatter_info[:,1],s = scatter_info[:,2]/5,cmap = cm,vmin = -1,vmax = 1,alpha = 0.6)
sc_cbar = plt.colorbar(sc,shrink=1,extend = 'both')
sc_cbar.set_label(label='Size Preference',size=12)
sc_cbar.set_ticks([-1,-0.5,0,0.5,1])
sc_cbar.set_ticklabels(['-1','-0.5','0','0.5','1'])
plt.xlabel('Score of PC(motor)',fontsize = 12)
plt.xlim(-0.05,1.05)
plt.xticks([0,0.5,1.0])
plt.title('Cluster Distribution in HSA dataset(PC)')
plt.show()

In [ ]:

fig,ax1 = plt.subplots(1,1,figsize = (3.3,1),dpi=800)

temp_df = df_active_neuron_normed_ratio
cluster_idx = np.unique(temp_df.cluster)
scatter_info = np.zeros((len(cluster_idx),4))
temp_np = df_active_neuron_normed_ratio.iloc[:,:6].to_numpy()
for cluster_i in range(0,len(cluster_idx)):
    i = int(cluster_idx[cluster_i])
    locals()['cluster_'+str(i)+'_roi_idx'] = list(np.where(temp_df.cluster == i)[0])
    scatter_info[cluster_i,0] = np.average(temp_df.iloc[locals()['cluster_'+str(i)+'_roi_idx'],-2])
    average_temp_cluster = np.average(temp_np[locals()['cluster_'+str(i)+'_roi_idx'],:4],axis = 0)
    scatter_info[cluster_i,1] = (average_temp_cluster[2]-average_temp_cluster[1])/(average_temp_cluster[2]+average_temp_cluster[1])
    scatter_info[cluster_i,2] = len(locals()['cluster_'+str(i)+'_roi_idx'])
    scatter_info[cluster_i,3] = int(i)
scatter_info[:,2] = scatter_info[:,2]/np.sum(scatter_info[:,2])    
for cluster_i in range(0,len(cluster_idx)):  
    i = int(cluster_idx[cluster_i])
#     if i==100:
#         ax1.annotate(str(i),(scatter_info[cluster_i,1]-0.025,0),fontsize = 4)
    if scatter_info[cluster_i,2]>.04:
        ax1.annotate(str(i),(scatter_info[cluster_i,1]-0.015,0),fontsize = 4)
sc = plt.scatter(scatter_info[:,1],np.zeros(scatter_info.shape[0]),c = scatter_info[:,1],s = scatter_info[:,2]*1000,cmap = cm,vmin = -1,vmax = 1,alpha = 0.6)

sc_cbar = plt.colorbar(sc,shrink=1,extend = 'both',orientation = 'horizontal',location = 'top',pad = 0.05)
sc_cbar.set_label(label='Size Preference',size=12)
sc_cbar.set_ticks([-1,-0.5,0,0.5,1])
sc_cbar.set_ticklabels(['-1(4°)','-0.5','0','0.5','1(15°)'])
plt.xlabel('Size Preference Index',fontsize = 12)
plt.xlim(-1.05,1.05)
# plt.ylim(-0.5,1.5)
plt.yticks([0.0],['HSA neurons'])
plt.show()

fig,ax1 = plt.subplots(1,1,figsize = (3.3,1),dpi=800)

df_active_pixel_normed_ratio = pd.read_csv('C:\\Users\\yxton\\Documents\\Code\\2p\\freezing_project\\clustering_files_RGC\\df_normed_ratio_ath5.csv')
df_active_pixel_normed_ratio_Freezing = df_active_pixel_normed_ratio[df_active_pixel_normed_ratio['cluster_type']!='PC']
cluster_idx_rgc = np.unique(df_active_pixel_normed_ratio.cluster)
scatter_info_rgc = np.zeros((len(cluster_idx_rgc),4))
for cluster_i in range(0,len(cluster_idx_rgc)):
    i = int(cluster_idx_rgc[cluster_i])
    locals()['cluster_'+str(i)+'_roi_idx'] = list(np.where(df_active_pixel_normed_ratio.cluster == i)[0])
    scatter_info_rgc[cluster_i,0] = np.average(df_active_pixel_normed_ratio.iloc[locals()['cluster_'+str(i)+'_roi_idx'],-2])
    average_temp_cluster = np.average(df_active_pixel_normed_ratio.iloc[locals()['cluster_'+str(i)+'_roi_idx'],8:12],axis = 0)
    scatter_info_rgc[cluster_i,1] = (average_temp_cluster[2]-average_temp_cluster[1])/(average_temp_cluster[2]+average_temp_cluster[1])
    scatter_info_rgc[cluster_i,2] = int(len(locals()['cluster_'+str(i)+'_roi_idx']))
    scatter_info_rgc[cluster_i,3] = int(i)
scatter_info_rgc[:,2] = scatter_info_rgc[:,2]/np.sum(scatter_info_rgc[:,2])    
for cluster_i in range(0,len(cluster_idx_rgc)):
    i = int(cluster_idx_rgc[cluster_i])
    if i==34:
        ax1.annotate(str(i),(scatter_info_rgc[cluster_i,1]-0.035,0),fontsize = 4)
    elif scatter_info_rgc[cluster_i,2]>.04:
        ax1.annotate(str(i),(scatter_info_rgc[cluster_i,1]-0.015,0),fontsize = 4)

sc = plt.scatter(scatter_info_rgc[:,1],np.zeros(scatter_info_rgc.shape[0]),c = scatter_info_rgc[:,1],s = scatter_info_rgc[:,2]*1000,cmap = cm,vmin = -1,vmax = 1,alpha = 0.6)


sc_cbar = plt.colorbar(sc,shrink=1,extend = 'both',orientation = 'horizontal',location = 'top',pad = 0.05)
sc_cbar.set_label(label='Size Preference',size=12)
sc_cbar.set_ticks([-1,-0.5,0,0.5,1])
sc_cbar.set_ticklabels(['-1(4°)','-0.5','0','0.5','1(15°)'])
plt.xlabel('Size Preference Index',fontsize = 12)
plt.xlim(-1.05,1.05)
# plt.ylim(-0.5,1.5)
plt.yticks([0.0],['RGC pixels'])
plt.show()


In [ ]:
# Sensory-motor index comparison between hsa neurons and rgc pixels
fig,ax1 = plt.subplots(1,1,figsize = (4,1.4),dpi=800)
scatter_info = np.zeros((len(cluster_idx_Freezing),4))
temp_df = df_active_neuron_normed_ratio_Freezing
temp_np = df_active_neuron_normed_ratio_Freezing.iloc[:,:6].to_numpy()
for cluster_i in range(0,len(cluster_idx_Freezing)):
    i = int(cluster_idx_Freezing[cluster_i])
    locals()['cluster_'+str(i)+'_roi_idx'] = list(np.where(temp_df.cluster == i)[0])
    scatter_info[cluster_i,0] = np.average(temp_df.iloc[locals()['cluster_'+str(i)+'_roi_idx'],-2])
    average_temp_cluster = np.average(temp_np[locals()['cluster_'+str(i)+'_roi_idx'],:4],axis = 0)
    scatter_info[cluster_i,1] = (average_temp_cluster[2]-average_temp_cluster[1])/(average_temp_cluster[2]+average_temp_cluster[1])
    scatter_info[cluster_i,2] = len(locals()['cluster_'+str(i)+'_roi_idx'])
    scatter_info[cluster_i,3] = int(i)
scatter_info[:,2] = scatter_info[:,2]/np.sum(scatter_info[:,2])    
for cluster_i in range(0,len(cluster_idx_Freezing)):  
    i = int(cluster_idx_Freezing[cluster_i])
    if i==100:
        ax1.annotate(str(i),(scatter_info[cluster_i,0]-0.025,0.2),fontsize = 4)
    elif scatter_info[cluster_i,2]>.1:
        ax1.annotate(str(i),(scatter_info[cluster_i,0]-0.015,0),fontsize = 4)
sc = plt.scatter(scatter_info[:,0],np.zeros(scatter_info.shape[0]),c = scatter_info[:,1],s = scatter_info[:,2]*1000,cmap = cm,vmin = -1,vmax = 1,alpha = 0.6)

df_active_pixel_normed_ratio = pd.read_csv('C:\\Users\\yxton\\Documents\\Code\\2p\\freezing_project\\clustering_files_RGC\\df_normed_ratio_ath5.csv')
df_active_pixel_normed_ratio_Freezing = df_active_pixel_normed_ratio[df_active_pixel_normed_ratio['cluster_type']!='PC']
cluster_idx_Freezing_rgc = np.unique(df_active_pixel_normed_ratio_Freezing.cluster)
scatter_info_rgc = np.zeros((len(cluster_idx_Freezing),4))
for cluster_i in range(0,len(cluster_idx_Freezing_rgc)):
    i = int(cluster_idx_Freezing_rgc[cluster_i])
    locals()['cluster_'+str(i)+'_roi_idx'] = list(np.where(df_active_pixel_normed_ratio.cluster == i)[0])
    scatter_info_rgc[cluster_i,0] = np.average(df_active_pixel_normed_ratio.iloc[locals()['cluster_'+str(i)+'_roi_idx'],-2])
    average_temp_cluster = np.average(df_active_pixel_normed_ratio.iloc[locals()['cluster_'+str(i)+'_roi_idx'],8:12],axis = 0)
    scatter_info_rgc[cluster_i,1] = (average_temp_cluster[2]-average_temp_cluster[1])/(average_temp_cluster[2]+average_temp_cluster[1])
    scatter_info_rgc[cluster_i,2] = int(len(locals()['cluster_'+str(i)+'_roi_idx']))
    scatter_info_rgc[cluster_i,3] = int(i)
scatter_info_rgc[:,2] = scatter_info_rgc[:,2]/np.sum(scatter_info_rgc[:,2])    
for cluster_i in range(0,len(cluster_idx_Freezing_rgc)):
    i = int(cluster_idx_Freezing_rgc[cluster_i])
    if i==34:
        ax1.annotate(str(i),(scatter_info_rgc[cluster_i,0]-0.035,1),fontsize = 4)
    elif scatter_info_rgc[cluster_i,2]>.08:
        ax1.annotate(str(i),(scatter_info_rgc[cluster_i,0]-0.015,1),fontsize = 4)

sc = plt.scatter(scatter_info_rgc[:,0],np.ones(scatter_info_rgc.shape[0]),c = scatter_info_rgc[:,1],s = scatter_info_rgc[:,2]*1000,cmap = cm,vmin = -1,vmax = 1,alpha = 0.6)


sc_cbar = plt.colorbar(sc,shrink=1,extend = 'both',orientation = 'horizontal',location = 'top',pad = 0.05)
sc_cbar.set_label(label='Size Preference',size=12)
sc_cbar.set_ticks([-1,-0.5,0,0.5,1])
sc_cbar.set_ticklabels(['-1(4°)','-0.5','0','0.5','1(15°)'])
plt.xlabel('Score of Freezing(motor)',fontsize = 12)
plt.xlim(-0.05,1.05)
plt.ylim(-0.5,1.5)
plt.yticks([0.0,1.0],['HSA neurons','RGC pixels'])
# plt.title('Cluster Distribution in HSA dataset(Freezing)')
plt.show()


In [ ]:
scatter_info[1,2] = scatter_info[1,2]/np.sum(scatter_info[:,2])

In [ ]:
scatter_info

In [ ]:
df_active_pixel_normed_ratio = pd.read_csv('C:\\Users\\yxton\\Documents\\Code\\2p\\freezing_project\\clustering_files_RGC\\df_normed_ratio_ath5.csv')
df_active_pixel_normed_ratio.iloc[:,8:12]

In [ ]:
scatter_info[:,2] = scatter_info[:,2]/np.sum(scatter_info[:,2])
scatter_info

In [ ]:
temp_df.iloc[locals()['cluster_'+str(i)+'_roi_idx'],:4]

In [ ]:
def Plot_Overlaid_Activity(ROI_iloc_list):
#     locals()[Fish_Index +'Max_DeltaF'] = pandas.read_excel(Saving_Directory + Fish_Index + '_Max_DeltaF2' + '.xlsx',sheet_name='Max_F')
#     locals()[Fish_Index +'Max_DeltaF'].drop(locals()[Fish_Index +'Max_DeltaF'].columns[0], inplace=True, axis=1)

#     Stimulus_Sizes = [2,3,4,6,8,10,0]
#     selective_clustering_data
    ROI_Activity_AVG = []
    ROI_Activity_STD = []
#         ROI_Activity = pandas.DataFrame()
#         for n in ROI_list:
#             if vsinfo.Right[n] == Stimulus_Sizes[s]:
#                 Trial = pandas.read_excel(Saving_Directory + Fish_Index + '_Trial' + str(n+1) + '.xlsx')
#                 ROI_Activity = ROI_Activity.append(Trial[Trial.Label == ROI])
    total_frame_no = np_trace_filtered.shape[1]
    for frame in range(0,total_frame_no):
        ROI_Activity_AVG.append(np_trace_filtered[ROI_iloc_list, frame].mean())
        ROI_Activity_STD.append(np_trace_filtered[ROI_iloc_list, frame].std())

#     # plot
#     Onset = 39
#     move = 46
#     stop = 54

    fig, ax = plt.subplots(1, 1, figsize=(15, 3))
#     plt.title('Averged Activity of ROI ' + str(ROI) + ' to different dot sizes', fontsize=20)
#     ax.set_ylabel('DeltaF/F', fontsize=15)
#     ax.set_xlabel('Frames', fontsize=16)
    ax.set_ylabel('DeltaF/F', fontsize=16)
    plt.ylim(-0.2, 3)
    plt.xlim(0, 39*6)
    plt.axvline(39, color='grey',alpha = 0.3)
    plt.axvline(39*2, color='grey',alpha = 0.3)
    plt.axvline(39*3, color='grey',alpha = 0.3)
    plt.axvline(39*4, color='grey',alpha = 0.3)
    plt.axvline(39*5, color='grey',alpha = 0.3)

    
#     plt.fill_between(np.arange(27,30),3,-0.2,linewidth = 1,color = 'lightcoral',alpha = 0.2)
    plt.fill_between(np.arange(27+39,30+39),3,-0.2,linewidth = 1,color = 'lightcoral',alpha = 0.2)
    plt.fill_between(np.arange(27+39*2,30+39*2),3,-0.2,linewidth = 1,color = 'lightcoral',alpha = 0.2)
    plt.fill_between(np.arange(27+39*3,30+39*3),3,-0.2,linewidth = 1,color = 'lightcoral',alpha = 0.2)
    plt.fill_between(np.arange(27+39*4,30+39*4),3,-0.2,linewidth = 1,color = 'lightcoral',alpha = 0.2)
    plt.fill_between(np.arange(27+39*5,30+39*5),3,-0.2,linewidth = 1,color = 'lightcoral',alpha = 0.2)

#     plt.axvline(Onset, 0, 100, label='Onset', color='grey')
#     plt.axvline(move, 0, 100, label='Move', color='red')
#     plt.axvline(stop, 0, 100, label='Stop', color='grey')
#     plt.axvline(stop + 8, 0, 100, label='1s after Stop', color='orange')
#     plt.axhline(0, 0, 121, label='AVG_Baseline', color='purple')
#     ax.tick_params(axis='x', labelsize=15)
#     ax.tick_params(axis='y', labelsize=15)
#     x_ticks = numpy.arange(0, 92, 7.7)
#     plt.xticks(x_ticks)
    

    ROI_Activity_AVG = np.array(ROI_Activity_AVG)
    ROI_Activity_STD = np.array(ROI_Activity_STD)
    yhat = ROI_Activity_AVG
    y_minus = ROI_Activity_AVG - ROI_Activity_STD
    y_plus = ROI_Activity_AVG + ROI_Activity_STD
#     bp = ax.plot(yhat, color='C',  linewidth=3)
    for i in ROI_iloc_list:
        ax.plot(np_trace_filtered[i,:],color = 'silver',linewidth=0.4,alpha = 0.25)
    plt.fill_between(range(0, total_frame_no), y_minus, y_plus, alpha=0.3, facecolor='c')
    bp = ax.plot(yhat, color='c',  linewidth=3)
#     bp = ax.plot(locals()['ROI_Activity_AVG_'+ str(Stimulus_Sizes[6])], color='C', label= str(Stimulus_Sizes[6])+ ' Degree', linewidth=3)
    
#     if locals()[Fish_Index +'Max_DeltaF'][locals()[Fish_Index +'Max_DeltaF'].Label == ROI].Side_is_Left.tolist()[0] == True:
#         Side = 'Ipsilateral to Stimulus'
#     else:
#         Side = 'Contralateral to Stimulus'
    
#     plt.plot([0],[0], color='black', label = 'Side = ' + Side )
#     plt.plot([0],[0], color='black', label = 'Size_Preference = ' + str(format(locals()[Fish_Index +'Max_DeltaF'][locals()[Fish_Index +'Max_DeltaF'].Label == ROI].Size_Pref_Index.tolist()[0],'.2f')))
#     plt.legend()
#     plt.setp(ax.get_xticklabels(), visible=False)
#     plt.setp(ax.get_yticklabels(), visible=False)
    plt.xticks([19,58,97,136,175,214])
    plt.yticks(fontsize = 14)
    ax.set_xticklabels(['No Stim', '4°', '15°', 'Luminance', 'Freezing (motor)', 'Prey (motor)'],fontsize = 18)
    ax.tick_params(axis='x', which='both', length=0)
#     ax.axis('off')
#     plt.show()
    return yhat

In [ ]:
np_trace_filtered

In [ ]:
cluster_idx=np.unique(clusters)
    
for i in cluster_idx:
    i = int(i)
    locals()['cluster_'+str(i)+'_roi_idx'] = list(np.where(clusters == i)[0])
    if len(locals()['cluster_'+str(i)+'_roi_idx'])>10:
        fig,ax1 = plt.subplots(1,1,figsize = (18,3))
        sns.heatmap(npy_VMV_filtered_cluster[locals()['cluster_'+str(i)+'_roi_idx'],:6],cmap = 'plasma',cbar_kws={'label':'Averaged Visomotor Vector Score','pad':0.02},ax = ax1,vmin = -0.3,vmax = 0.7)
        ax1.set_xticklabels(['No Stim', '4°', '15°', 'Luminance', 'Freezing (motor)', 'Prey (motor)'],fontsize = 18)
        ax1.figure.axes[-1].yaxis.label.set_size(12)
        ax1.figure.axes[-1].tick_params(labelsize=14)
        for j in range(0,4):
            ax1.get_xticklabels()[j].set_color("maroon")
        for k in range(4,6):
            ax1.get_xticklabels()[k].set_color("navy")
        ax1.set_yticks([]) 
    #     plt.title('Averaged activity of cluster'+str(i)+', n = '+str(len(locals()['cluster_'+str(i)+'_roi_idx'])),fontsize = 16)
        plt.title('Averaged activity of cluster'+str(i),fontsize = 22)
        plt.show()

        # the averaged trace of each cluster
        Plot_Overlaid_Activity(locals()['cluster_'+str(i)+'_roi_idx'])
        plt.title('Averaged activity of cluster'+str(i)+', n = '+str(len(locals()['cluster_'+str(i)+'_roi_idx'])),fontsize = 22)
        plt.show()

In [ ]:
cluster_idx=np.unique(clusters)
    
for i in cluster_idx:
    i = int(i)
    locals()['cluster_'+str(i)+'_roi_idx'] = list(np.where(clusters == i)[0])
    if len(locals()['cluster_'+str(i)+'_roi_idx'])>10:
        fig,ax1 = plt.subplots(1,1,figsize = (18,3))
        sns.heatmap(npy_VMV_filtered_cluster[locals()['cluster_'+str(i)+'_roi_idx'],:6],cmap = 'plasma',cbar_kws={'label':'Averaged Visomotor Vector Score','pad':0.02},ax = ax1,vmin = -0.3,vmax = 0.7)
        ax1.set_xticklabels(['No Stim', '4°', '15°', 'Luminance', 'Freezing (motor)', 'Prey (motor)'],fontsize = 18)
        ax1.figure.axes[-1].yaxis.label.set_size(12)
        ax1.figure.axes[-1].tick_params(labelsize=14)
        for j in range(0,4):
            ax1.get_xticklabels()[j].set_color("maroon")
        for k in range(4,6):
            ax1.get_xticklabels()[k].set_color("navy")
        ax1.set_yticks([]) 
    #     plt.title('Averaged activity of cluster'+str(i)+', n = '+str(len(locals()['cluster_'+str(i)+'_roi_idx'])),fontsize = 16)
        plt.title('Averaged activity of cluster'+str(i),fontsize = 22)
        plt.show()

        # the averaged trace of each cluster
        Plot_Overlaid_Activity(locals()['cluster_'+str(i)+'_roi_idx'])
        plt.title('Averaged activity of cluster'+str(i)+', n = '+str(len(locals()['cluster_'+str(i)+'_roi_idx'])),fontsize = 22)
        plt.show()

In [ ]:
import matplotlib.cm as cmx
import matplotlib as mpl
fig, ax = plt.subplots(figsize=(1.3, 5),dpi=800)
fig.subplots_adjust(right=0.5)

c_map = plt.get_cmap('plasma')
cNorm = colors.Normalize(vmin = -0.3,vmax = 0.7)
scalarMap = cmx.ScalarMappable(norm=cNorm, cmap=c_map)  


cb1 = mpl.colorbar.ColorbarBase(ax, cmap=c_map,
                                norm=cNorm,
                                orientation='vertical')
cb1.set_label('Averaged VMV Score',fontsize = 27)
cb1.ax.tick_params(labelsize=23)
plt.show()


In [ ]:
# example cluster illustration
temp_cluster_id  = 13

fig,axes = plt.subplots(2,1,figsize = (9,4),dpi = 800)
locals()['cluster_'+str(temp_cluster_id)+'_roi_idx'] = list(np.where(clusters == temp_cluster_id)[0])
sns.heatmap(npy_VMV_filtered_cluster[locals()['cluster_'+str(temp_cluster_id)+'_roi_idx'],:6],cmap = 'plasma',cbar_kws={'label':'Averaged Visomotor Vector Score','pad':0.02},ax = axes[0],cbar = False,vmin = -0.3,vmax = 0.7)
axes[0].set_xticklabels(['No Stim', '4°', '15°', 'Luminance', 'Freezing (motor)', 'Prey (motor)'],fontsize = 12)
# axes[0].figure.axes[-1].yaxis.label.set_size(10)
# axes[0].figure.axes[-1].tick_params(labelsize=10)
for j in range(0,4):
    axes[0].get_xticklabels()[j].set_color("maroon")
for k in range(4,6):
    axes[0].get_xticklabels()[k].set_color("navy")
axes[0].set_yticks([]) 

ROI_Activity_AVG = []
ROI_Activity_STD = []
total_frame_no = np_trace_filtered.shape[1]
for frame in range(0,total_frame_no):
    ROI_Activity_AVG.append(np_trace_filtered[locals()['cluster_'+str(temp_cluster_id)+'_roi_idx'], frame].mean())
    ROI_Activity_STD.append(np_trace_filtered[locals()['cluster_'+str(temp_cluster_id)+'_roi_idx'], frame].std())

axes[1].set_ylabel('DeltaF/F', fontsize=12)
axes[1].set_ylim(-0.2, 3)
axes[1].set_xlim(0, 39*6)
axes[1].axvline(39, color='grey',alpha = 0.3)
axes[1].axvline(39*2, color='grey',alpha = 0.3)
axes[1].axvline(39*3, color='grey',alpha = 0.3)
axes[1].axvline(39*4, color='grey',alpha = 0.3)
axes[1].axvline(39*5, color='grey',alpha = 0.3)

axes[1].fill_between(np.arange(27+39,29+39),3,-0.2,linewidth = 1,color = 'violet',alpha = 0.3)
axes[1].fill_between(np.arange(27+39*2,29+39*2),3,-0.2,linewidth = 1,color = 'violet',alpha = 0.3)
axes[1].fill_between(np.arange(27+39*3,29+39*3),3,-0.2,linewidth = 1,color = 'violet',alpha = 0.3)
axes[1].fill_between(np.arange(27+39*4,29+39*4),3,-0.2,linewidth = 1,color = 'violet',alpha = 0.3)
axes[1].fill_between(np.arange(27+39*5,29+39*5),3,-0.2,linewidth = 1,color = 'violet',alpha = 0.3)



ROI_Activity_AVG = np.array(ROI_Activity_AVG)
ROI_Activity_STD = np.array(ROI_Activity_STD)
yhat = ROI_Activity_AVG
y_minus = ROI_Activity_AVG - ROI_Activity_STD
y_plus = ROI_Activity_AVG + ROI_Activity_STD
for i in locals()['cluster_'+str(temp_cluster_id)+'_roi_idx']:
    axes[1].plot(np_trace_filtered[i,:],color = 'silver',linewidth=0.4,alpha = 0.25)
axes[1].fill_between(range(0, total_frame_no), y_minus, y_plus, alpha=0.3, facecolor='c')
bp = axes[1].plot(yhat, color='dodgerblue',  linewidth=3)
axes[1].set_xticks([19,58,97,136,175,214])
axes[1].set_yticks([0,1,2,3])
axes[1].set_xticklabels(['No Stim', '4°', '15°', 'Luminance', 'Freezing (motor)', 'Prey (motor)'],fontsize = 12)
axes[1].tick_params(axis='x', which='both', length=0)

fig.tight_layout()
plt.show()


# anatomical

print('Cluster'+str(temp_cluster_id))
colorcode = 'dodgerblue'
size = 1.5

xy_cropping = [200,559,60,517]
xz_cropping = [0,359,60,517]
yz_cropping = [0,359,120,577]

xy = np.mean(atlas,axis = 0)
yz = np.mean(atlas,axis = 2)
xz = np.mean(atlas,axis = 1)
rotated_yz = np.zeros((yz.shape[0],yz.shape[1]))
rotated_xz = np.zeros((xz.shape[0],xz.shape[1]))

for z in range(0,yz.shape[0]):
    rotated_yz[z,:] = yz[yz.shape[0]-1-z,:]
    rotated_xz[z,:] = xz[xz.shape[0]-1-z,:]


X = df_filtered_reg_clusters[df_filtered_reg_clusters['cluster']==temp_cluster_id].x/0.994
Y = df_filtered_reg_clusters[df_filtered_reg_clusters['cluster']==temp_cluster_id].y/0.994
Z = df_filtered_reg_clusters[df_filtered_reg_clusters['cluster']==temp_cluster_id].z

filtered_idx = []
for x,y,z,i in zip(X,Y,Z,range(0,X.shape[0])):
    if mask_xy[int(y),int(x)] == 255 and mask_xz[int(z),int(y)] == 255 and mask_yz[int(z),int(x)] == 255:
        filtered_idx.append(i)


fig,axes = plt.subplots(1, 3,figsize = (8,3),subplot_kw=dict(box_aspect=3.59/4.57),dpi=800)

ax = sns.heatmap(xy[xy_cropping[0]:xy_cropping[1],xy_cropping[2]:xy_cropping[3]],ax = axes[0],cmap = 'gray',cbar = False,vmin = 0,vmax = 30)
axes[0].scatter(np.array(X)[filtered_idx]-xy_cropping[2],np.array(Y)[filtered_idx]-xy_cropping[0],color = colorcode,s = size)
axes[0].axis('off')


ax = sns.heatmap(rotated_xz[xz_cropping[0]:xz_cropping[1],xz_cropping[2]:xz_cropping[3]],ax = axes[1],cmap = 'gray',cbar = False,vmin = 0,vmax = 30)
axes[1].scatter(np.array(X)[filtered_idx]-xz_cropping[2],358-np.array(Z)[filtered_idx]-xz_cropping[0],color = colorcode,s = size)
axes[1].axis('off')



ax = sns.heatmap(rotated_yz[yz_cropping[0]:yz_cropping[1],yz_cropping[2]:yz_cropping[3]],ax = axes[2],cmap = 'gray',cbar = False,vmin = 0,vmax = 30)
axes[2].scatter(np.array(Y)[filtered_idx]-yz_cropping[2],358-np.array(Z)[filtered_idx]-yz_cropping[0],color = colorcode,s = size)
axes[2].axis('off')

cols = ['Horizontal','Coronal','Sagittal']
pad = 5 
for ax, col in zip(axes, cols):
    ax.annotate(col, xy=(0.5, 1.04), xytext=(0, pad),
                xycoords='axes fraction', textcoords='offset points',
                size=15, ha='center', va='baseline')

# 30 micron scale bar
axes[1].axhline(y = 340,xmin = 0.8,xmax = 0.8+30.18/457,color = 'white',linewidth = 2)

fig.tight_layout()
plt.show()

In [ ]:
# example cluster illustration
temp_cluster_id  = 73

fig,axes = plt.subplots(2,1,figsize = (9,4),dpi = 800)
locals()['cluster_'+str(temp_cluster_id)+'_roi_idx'] = list(np.where(clusters == temp_cluster_id)[0])
sns.heatmap(npy_VMV_filtered_cluster[locals()['cluster_'+str(temp_cluster_id)+'_roi_idx'],:6],cmap = 'plasma',cbar_kws={'label':'Averaged Visomotor Vector Score','pad':0.02},ax = axes[0],cbar = False,vmin = -0.3,vmax = 0.7)
axes[0].set_xticklabels(['No Stim', '4°', '15°', 'Luminance', 'Freezing (motor)', 'Prey (motor)'],fontsize = 12)
# axes[0].figure.axes[-1].yaxis.label.set_size(10)
# axes[0].figure.axes[-1].tick_params(labelsize=10)
for j in range(0,4):
    axes[0].get_xticklabels()[j].set_color("maroon")
for k in range(4,6):
    axes[0].get_xticklabels()[k].set_color("navy")
axes[0].set_yticks([]) 

ROI_Activity_AVG = []
ROI_Activity_STD = []
total_frame_no = np_trace_filtered.shape[1]
for frame in range(0,total_frame_no):
    ROI_Activity_AVG.append(np_trace_filtered[locals()['cluster_'+str(temp_cluster_id)+'_roi_idx'], frame].mean())
    ROI_Activity_STD.append(np_trace_filtered[locals()['cluster_'+str(temp_cluster_id)+'_roi_idx'], frame].std())

axes[1].set_ylabel('DeltaF/F', fontsize=12)
axes[1].set_ylim(-0.2, 3)
axes[1].set_xlim(0, 39*6)
axes[1].axvline(39, color='grey',alpha = 0.3)
axes[1].axvline(39*2, color='grey',alpha = 0.3)
axes[1].axvline(39*3, color='grey',alpha = 0.3)
axes[1].axvline(39*4, color='grey',alpha = 0.3)
axes[1].axvline(39*5, color='grey',alpha = 0.3)

    
axes[1].fill_between(np.arange(27+39,29+39),3,-0.2,linewidth = 1,color = 'violet',alpha = 0.3)
axes[1].fill_between(np.arange(27+39*2,29+39*2),3,-0.2,linewidth = 1,color = 'violet',alpha = 0.3)
axes[1].fill_between(np.arange(27+39*3,29+39*3),3,-0.2,linewidth = 1,color = 'violet',alpha = 0.3)
axes[1].fill_between(np.arange(27+39*4,29+39*4),3,-0.2,linewidth = 1,color = 'violet',alpha = 0.3)
axes[1].fill_between(np.arange(27+39*5,29+39*5),3,-0.2,linewidth = 1,color = 'violet',alpha = 0.3)



ROI_Activity_AVG = np.array(ROI_Activity_AVG)
ROI_Activity_STD = np.array(ROI_Activity_STD)
yhat = ROI_Activity_AVG
y_minus = ROI_Activity_AVG - ROI_Activity_STD
y_plus = ROI_Activity_AVG + ROI_Activity_STD
for i in locals()['cluster_'+str(temp_cluster_id)+'_roi_idx']:
    axes[1].plot(np_trace_filtered[i,:],color = 'silver',linewidth=0.4,alpha = 0.25)
axes[1].fill_between(range(0, total_frame_no), y_minus, y_plus, alpha=0.3, facecolor='c')
bp = axes[1].plot(yhat, color='crimson',  linewidth=3)
axes[1].set_xticks([19,58,97,136,175,214])
axes[1].set_yticks([0,1,2,3])
axes[1].set_xticklabels(['No Stim', '4°', '15°', 'Luminance', 'Freezing (motor)', 'Prey (motor)'],fontsize = 12)
axes[1].tick_params(axis='x', which='both', length=0)

fig.tight_layout()
plt.show()


# anatomical

print('Cluster'+str(temp_cluster_id))
colorcode = 'crimson'
size = 1.5

xy_cropping = [200,559,60,517]
xz_cropping = [0,359,60,517]
yz_cropping = [0,359,120,577]

xy = np.mean(atlas,axis = 0)
yz = np.mean(atlas,axis = 2)
xz = np.mean(atlas,axis = 1)
rotated_yz = np.zeros((yz.shape[0],yz.shape[1]))
rotated_xz = np.zeros((xz.shape[0],xz.shape[1]))

for z in range(0,yz.shape[0]):
    rotated_yz[z,:] = yz[yz.shape[0]-1-z,:]
    rotated_xz[z,:] = xz[xz.shape[0]-1-z,:]


X = df_filtered_reg_clusters[df_filtered_reg_clusters['cluster']==temp_cluster_id].x/0.994
Y = df_filtered_reg_clusters[df_filtered_reg_clusters['cluster']==temp_cluster_id].y/0.994
Z = df_filtered_reg_clusters[df_filtered_reg_clusters['cluster']==temp_cluster_id].z

filtered_idx = []
for x,y,z,i in zip(X,Y,Z,range(0,X.shape[0])):
    if mask_xy[int(y),int(x)] == 255 and mask_xz[int(z),int(y)] == 255 and mask_yz[int(z),int(x)] == 255:
        filtered_idx.append(i)


fig,axes = plt.subplots(1, 3,figsize = (8,3),subplot_kw=dict(box_aspect=3.59/4.57),dpi=800)

ax = sns.heatmap(xy[xy_cropping[0]:xy_cropping[1],xy_cropping[2]:xy_cropping[3]],ax = axes[0],cmap = 'gray',cbar = False,vmin = 0,vmax = 30)
axes[0].scatter(np.array(X)[filtered_idx]-xy_cropping[2],np.array(Y)[filtered_idx]-xy_cropping[0],color = colorcode,s = size)
axes[0].axis('off')


ax = sns.heatmap(rotated_xz[xz_cropping[0]:xz_cropping[1],xz_cropping[2]:xz_cropping[3]],ax = axes[1],cmap = 'gray',cbar = False,vmin = 0,vmax = 30)
axes[1].scatter(np.array(X)[filtered_idx]-xz_cropping[2],358-np.array(Z)[filtered_idx]-xz_cropping[0],color = colorcode,s = size)
axes[1].axis('off')



ax = sns.heatmap(rotated_yz[yz_cropping[0]:yz_cropping[1],yz_cropping[2]:yz_cropping[3]],ax = axes[2],cmap = 'gray',cbar = False,vmin = 0,vmax = 30)
axes[2].scatter(np.array(Y)[filtered_idx]-yz_cropping[2],358-np.array(Z)[filtered_idx]-yz_cropping[0],color = colorcode,s = size)
axes[2].axis('off')

cols = ['Horizontal','Coronal','Sagittal']
pad = 5 
for ax, col in zip(axes, cols):
    ax.annotate(col, xy=(0.5, 1.04), xytext=(0, pad),
                xycoords='axes fraction', textcoords='offset points',
                size=15, ha='center', va='baseline')

# 30 micron scale bar
axes[1].axhline(y = 340,xmin = 0.8,xmax = 0.8+30.18/457,color = 'white',linewidth = 2)

fig.tight_layout()
plt.show()


In [ ]:
atlas_path = 'C:\\Users\\yxton\\Documents\\Code\\registration_code_HSA\\T_AVG_H2BGCaMP.tif'
atlas = tifffile.imread(atlas_path)

In [ ]:
mask_xy_path = 'C:\\Users\\yxton\\Documents\\Code\\2p\\freezing_project\\clustering_files\\Mask_xy.tif'
mask_xy = tifffile.imread(mask_xy_path)
mask_xz_path = 'C:\\Users\\yxton\\Documents\\Code\\2p\\freezing_project\\clustering_files\\Mask_xz.tif'
mask_xz = tifffile.imread(mask_xz_path)
mask_yz_path = 'C:\\Users\\yxton\\Documents\\Code\\2p\\freezing_project\\clustering_files\\Mask_yz.tif'
mask_yz = tifffile.imread(mask_yz_path)

In [ ]:
# example cluster anatomical illustration


xy_cropping = [200,559,0,597]
xz_cropping = [0,359,0,597]
yz_cropping = [0,359,0,597]

xy = np.mean(atlas,axis = 0)
yz = np.mean(atlas,axis = 2)
xz = np.mean(atlas,axis = 1)
rotated_yz = np.zeros((yz.shape[0],yz.shape[1]))
rotated_xz = np.zeros((xz.shape[0],xz.shape[1]))

for z in range(0,yz.shape[0]):
    rotated_yz[z,:] = yz[yz.shape[0]-1-z,:]
    rotated_xz[z,:] = xz[xz.shape[0]-1-z,:]

    
fig,axes = plt.subplots(1, 3,figsize = (9,3),subplot_kw=dict(box_aspect=3.59/5.97),dpi=800)

ax = sns.heatmap(xy[xy_cropping[0]:xy_cropping[1],xy_cropping[2]:xy_cropping[3]],ax = axes[0],cmap = 'gray',cbar = False,vmin = 0,vmax = 30)
ax = sns.heatmap(rotated_xz[xz_cropping[0]:xz_cropping[1],xz_cropping[2]:xz_cropping[3]],ax = axes[1],cmap = 'gray',cbar = False,vmin = 0,vmax = 30)
ax = sns.heatmap(rotated_yz[yz_cropping[0]:yz_cropping[1],yz_cropping[2]:yz_cropping[3]],ax = axes[2],cmap = 'gray',cbar = False,vmin = 0,vmax = 30)

size = 1

# colorcode = '#50d4f0'
# colorcode = 'lawngreen'
# colorcode = 'deepskyblue'
colorcode = 'dodgerblue'

freezing_cluster_id_list = [13,9,105]
for temp_cluster_id in freezing_cluster_id_list:
    X = df_filtered_reg_clusters[df_filtered_reg_clusters['cluster']==temp_cluster_id].x/0.994
    Y = df_filtered_reg_clusters[df_filtered_reg_clusters['cluster']==temp_cluster_id].y/0.994
    Z = df_filtered_reg_clusters[df_filtered_reg_clusters['cluster']==temp_cluster_id].z
    filtered_idx = []
    for x,y,z,i in zip(X,Y,Z,range(0,X.shape[0])):
        if mask_xy[int(y),int(x)] == 255 and mask_xz[int(z),int(y)] == 255 and mask_yz[int(z),int(x)] == 255:
            filtered_idx.append(i)

    axes[0].scatter(np.array(X)[filtered_idx]-xy_cropping[2],np.array(Y)[filtered_idx]-xy_cropping[0],color = colorcode,s = size,alpha = 0.6)
    axes[1].scatter(np.array(X)[filtered_idx]-xz_cropping[2],358-np.array(Z)[filtered_idx]-xz_cropping[0],color = colorcode,s = size,alpha = 0.6)
    axes[2].scatter(np.array(Y)[filtered_idx]-xz_cropping[2],358-np.array(Z)[filtered_idx]-xz_cropping[0],color = colorcode,s = size,alpha = 0.6)

pc_cluster_id_list = [77,79,73]
# colorcode = '#fc5b44'
# colorcode = 'magenta'
colorcode = 'crimson'

for temp_cluster_id in pc_cluster_id_list:
    X = df_filtered_reg_clusters[df_filtered_reg_clusters['cluster']==temp_cluster_id].x/0.994
    Y = df_filtered_reg_clusters[df_filtered_reg_clusters['cluster']==temp_cluster_id].y/0.994
    Z = df_filtered_reg_clusters[df_filtered_reg_clusters['cluster']==temp_cluster_id].z

    filtered_idx = []
    for x,y,z,i in zip(X,Y,Z,range(0,X.shape[0])):
        if mask_xy[int(y),int(x)] == 255 and mask_xz[int(z),int(y)] == 255 and mask_yz[int(z),int(x)] == 255:
            filtered_idx.append(i)
    axes[0].scatter(np.array(X)[filtered_idx]-xy_cropping[2],np.array(Y)[filtered_idx]-xy_cropping[0],color = colorcode,s = size,alpha = 0.35)
    axes[1].scatter(np.array(X)[filtered_idx]-xz_cropping[2],358-np.array(Z)[filtered_idx]-xz_cropping[0],color = colorcode,s = size,alpha = 0.35)
    axes[2].scatter(np.array(Y)[filtered_idx]-xz_cropping[2],358-np.array(Z)[filtered_idx]-xz_cropping[0],color = colorcode,s = size,alpha = 0.35)





axes[0].axis('off')
axes[1].axis('off')
axes[2].axis('off')

cols = ['Horizontal','Coronal','Sagittal']
pad = 5 
for ax, col in zip(axes, cols):
    ax.annotate(col, xy=(0.5, 1.04), xytext=(0, pad),
                xycoords='axes fraction', textcoords='offset points',
                size=18, ha='center', va='baseline')

fig.tight_layout()
plt.show()


In [ ]:
# cropped version +scale bar


xy_cropping = [200,559,60,517]
xz_cropping = [0,359,60,517]
yz_cropping = [0,359,120,577]

xy = np.mean(atlas,axis = 0)
yz = np.mean(atlas,axis = 2)
xz = np.mean(atlas,axis = 1)
rotated_yz = np.zeros((yz.shape[0],yz.shape[1]))
rotated_xz = np.zeros((xz.shape[0],xz.shape[1]))

for z in range(0,yz.shape[0]):
    rotated_yz[z,:] = yz[yz.shape[0]-1-z,:]
    rotated_xz[z,:] = xz[xz.shape[0]-1-z,:]

    
fig,axes = plt.subplots(1, 3,figsize = (8,3),subplot_kw=dict(box_aspect=3.59/4.57),dpi=800)

ax = sns.heatmap(xy[xy_cropping[0]:xy_cropping[1],xy_cropping[2]:xy_cropping[3]],ax = axes[0],cmap = 'gray',cbar = False,vmin = 0,vmax = 30,square = True)
ax = sns.heatmap(rotated_xz[xz_cropping[0]:xz_cropping[1],xz_cropping[2]:xz_cropping[3]],ax = axes[1],cmap = 'gray',cbar = False,vmin = 0,vmax = 30,square = True)
ax = sns.heatmap(rotated_yz[yz_cropping[0]:yz_cropping[1],yz_cropping[2]:yz_cropping[3]],ax = axes[2],cmap = 'gray',cbar = False,vmin = 0,vmax = 30,square = True)

size = 1

# colorcode = '#50d4f0'
# colorcode = 'lawngreen'
# colorcode = 'deepskyblue'
colorcode = 'dodgerblue'

freezing_cluster_id_list = [13,9,105]
for temp_cluster_id in freezing_cluster_id_list:
    X = df_filtered_reg_clusters[df_filtered_reg_clusters['cluster']==temp_cluster_id].x/0.994
    Y = df_filtered_reg_clusters[df_filtered_reg_clusters['cluster']==temp_cluster_id].y/0.994
    Z = df_filtered_reg_clusters[df_filtered_reg_clusters['cluster']==temp_cluster_id].z
    filtered_idx = []
    for x,y,z,i in zip(X,Y,Z,range(0,X.shape[0])):
        if mask_xy[int(y),int(x)] == 255 and mask_xz[int(z),int(y)] == 255 and mask_yz[int(z),int(x)] == 255:
            filtered_idx.append(i)

    axes[0].scatter(np.array(X)[filtered_idx]-xy_cropping[2],np.array(Y)[filtered_idx]-xy_cropping[0],color = colorcode,s = size,alpha = 0.6)
    axes[1].scatter(np.array(X)[filtered_idx]-xz_cropping[2],358-np.array(Z)[filtered_idx]-xz_cropping[0],color = colorcode,s = size,alpha = 0.6)
    axes[2].scatter(np.array(Y)[filtered_idx]-yz_cropping[2],358-np.array(Z)[filtered_idx]-yz_cropping[0],color = colorcode,s = size,alpha = 0.6)

pc_cluster_id_list = [77,79,73]
# colorcode = '#fc5b44'
# colorcode = 'magenta'
colorcode = 'crimson'

for temp_cluster_id in pc_cluster_id_list:
    X = df_filtered_reg_clusters[df_filtered_reg_clusters['cluster']==temp_cluster_id].x/0.994
    Y = df_filtered_reg_clusters[df_filtered_reg_clusters['cluster']==temp_cluster_id].y/0.994
    Z = df_filtered_reg_clusters[df_filtered_reg_clusters['cluster']==temp_cluster_id].z

    filtered_idx = []
    for x,y,z,i in zip(X,Y,Z,range(0,X.shape[0])):
        if mask_xy[int(y),int(x)] == 255 and mask_xz[int(z),int(y)] == 255 and mask_yz[int(z),int(x)] == 255:
            filtered_idx.append(i)
    axes[0].scatter(np.array(X)[filtered_idx]-xy_cropping[2],np.array(Y)[filtered_idx]-xy_cropping[0],color = colorcode,s = size,alpha = 0.35)
    axes[1].scatter(np.array(X)[filtered_idx]-xz_cropping[2],358-np.array(Z)[filtered_idx]-xz_cropping[0],color = colorcode,s = size,alpha = 0.35)
    axes[2].scatter(np.array(Y)[filtered_idx]-yz_cropping[2],358-np.array(Z)[filtered_idx]-yz_cropping[0],color = colorcode,s = size,alpha = 0.35)


# 30 micron scale bar
axes[1].axhline(y = 340,xmin = 0.8,xmax = 0.8+30.18/457,color = 'white',linewidth = 2)


axes[0].axis('off')
axes[1].axis('off')
axes[2].axis('off')

cols = ['Horizontal','Coronal','Sagittal']
pad = 5 
for ax, col in zip(axes, cols):
    ax.annotate(col, xy=(0.5, 1.04), xytext=(0, pad),
                xycoords='axes fraction', textcoords='offset points',
                size=15, ha='center', va='baseline')

fig.tight_layout()
plt.show()


In [ ]:
# bar plot of neuron no. in each brain region
region_path_list = glob.glob('C:\\Users\\yxton\\Documents\\Code\\2p\\freezing_project\\clustering_files\\mapzebrain_brain_regions\\*tif')

region_list = []

for path in region_path_list:
    region = path.split('\\')[-1][:-4]
    locals()[region] = tifffile.imread(path)
    region_list.append(region)
    
#freezing_cluster_id_list pc_cluster_id_list
np_region_NeuronNumber = np.zeros((2,len(region_list)))
for temp_id in freezing_cluster_id_list:
    temp_id = int(temp_id)
    X = df_filtered_reg_clusters[df_filtered_reg_clusters['cluster']==temp_id].x/0.994
    Y = df_filtered_reg_clusters[df_filtered_reg_clusters['cluster']==temp_id].y/0.994
    Z = df_filtered_reg_clusters[df_filtered_reg_clusters['cluster']==temp_id].z
    
    filtered_idx = []
    for x,y,z,i in zip(X,Y,Z,range(0,X.shape[0])):
        if mask_xy[int(y),int(x)] == 255 and mask_xz[int(z),int(y)] == 255 and mask_yz[int(z),int(x)] == 255:
            filtered_idx.append(i)
            
    X = np.array(X)[filtered_idx]
    Y = np.array(Y)[filtered_idx]
    Z = np.array(Z)[filtered_idx]
    
    No_of_cells = X.shape[0]
    for j in range(0,len(region_list)):
        temp_no = 0
        temp_region = locals()[region_list[j]]
        for i in range(0,No_of_cells):
            if temp_region[int(Z[i]),int(Y[i]),int(X[i])] == 255:
                temp_no = temp_no+1
        np_region_NeuronNumber[0,j] = np_region_NeuronNumber[0,j]+temp_no
        
for temp_id in pc_cluster_id_list:
    temp_id = int(temp_id)
    X = df_filtered_reg_clusters[df_filtered_reg_clusters['cluster']==temp_id].x/0.994
    Y = df_filtered_reg_clusters[df_filtered_reg_clusters['cluster']==temp_id].y/0.994
    Z = df_filtered_reg_clusters[df_filtered_reg_clusters['cluster']==temp_id].z
    
    filtered_idx = []
    for x,y,z,i in zip(X,Y,Z,range(0,X.shape[0])):
        if mask_xy[int(y),int(x)] == 255 and mask_xz[int(z),int(y)] == 255 and mask_yz[int(z),int(x)] == 255:
            filtered_idx.append(i)
            
    X = np.array(X)[filtered_idx]
    Y = np.array(Y)[filtered_idx]
    Z = np.array(Z)[filtered_idx]
    
    No_of_cells = X.shape[0]
    for j in range(0,len(region_list)):
        temp_no = 0
        temp_region = locals()[region_list[j]]
        for i in range(0,No_of_cells):
            if temp_region[int(Z[i]),int(Y[i]),int(X[i])] == 255:
                temp_no = temp_no+1
        np_region_NeuronNumber[1,j] = np_region_NeuronNumber[1,j]+temp_no
        
labels = region_list

x = np.arange(len(labels)) 
width = 0.45

fig, ax = plt.subplots(figsize = (8,4),dpi = 800)
rects1 = ax.bar(x - width/2, np_region_NeuronNumber[0,:], width, label='15°Cluster',color = 'dodgerblue')
rects2 = ax.bar(x + width/2, np_region_NeuronNumber[1,:], width, label='4°Cluster',color = 'crimson')
ax.set_ylabel('No. of Neurons',fontsize = 18)
plt.ylim(0,1150)
plt.yticks(fontsize =15)
ax.set_xticks(x)
ax.set_xticklabels(labels,fontsize = 15)
plt.xlabel('Region',fontsize=18)
ax.legend(fontsize = 18)
ax.set_xticklabels(region_list, rotation = 40,fontsize = 15,horizontalalignment = 'right')

def autolabel(rects):
    for rect in rects:
        height = rect.get_height()
        ax.annotate('{}'.format(int(height)),
                    xy=(rect.get_x() + rect.get_width() / 2, height),
                    xytext=(0, 3),  # 3 points vertical offset
                    textcoords="offset points",
                    ha='center', va='bottom')


autolabel(rects1)
autolabel(rects2)

plt.show()

In [ ]:
# v2 bar plot of neuron no. in each brain region
region_path_list = glob.glob('C:\\Users\\yxton\\Documents\\Code\\2p\\freezing_project\\clustering_files\\mapzebrain_brain_regions\\*tif')
# region_path_list = [region_path_list[0]]+[region_path_list[2]]+region_path_list[4:6]
region_path_list = [region_path_list[5]]+[region_path_list[4]]+[region_path_list[0]]+[region_path_list[2]]
region_list = []

for path in region_path_list:
    region = path.split('\\')[-1][:-4]
    locals()[region] = tifffile.imread(path)
    region_list.append(region)
    
#freezing_cluster_id_list pc_cluster_id_list
np_region_NeuronNumber = np.zeros((2,len(region_list)))
for temp_id in freezing_cluster_id_list:
    temp_id = int(temp_id)
    X = df_filtered_reg_clusters[df_filtered_reg_clusters['cluster']==temp_id].x/0.994
    Y = df_filtered_reg_clusters[df_filtered_reg_clusters['cluster']==temp_id].y/0.994
    Z = df_filtered_reg_clusters[df_filtered_reg_clusters['cluster']==temp_id].z
    
    filtered_idx = []
    for x,y,z,i in zip(X,Y,Z,range(0,X.shape[0])):
        if mask_xy[int(y),int(x)] == 255 and mask_xz[int(z),int(y)] == 255 and mask_yz[int(z),int(x)] == 255:
            filtered_idx.append(i)
            
    X = np.array(X)[filtered_idx]
    Y = np.array(Y)[filtered_idx]
    Z = np.array(Z)[filtered_idx]
    
    No_of_cells = X.shape[0]
    for j in range(0,len(region_list)):
        temp_no = 0
        temp_region = locals()[region_list[j]]
        for i in range(0,No_of_cells):
            if temp_region[int(Z[i]),int(Y[i]),int(X[i])] == 255:
                temp_no = temp_no+1
        np_region_NeuronNumber[0,j] = np_region_NeuronNumber[0,j]+temp_no
        
for temp_id in pc_cluster_id_list:
    temp_id = int(temp_id)
    X = df_filtered_reg_clusters[df_filtered_reg_clusters['cluster']==temp_id].x/0.994
    Y = df_filtered_reg_clusters[df_filtered_reg_clusters['cluster']==temp_id].y/0.994
    Z = df_filtered_reg_clusters[df_filtered_reg_clusters['cluster']==temp_id].z
    
    filtered_idx = []
    for x,y,z,i in zip(X,Y,Z,range(0,X.shape[0])):
        if mask_xy[int(y),int(x)] == 255 and mask_xz[int(z),int(y)] == 255 and mask_yz[int(z),int(x)] == 255:
            filtered_idx.append(i)
            
    X = np.array(X)[filtered_idx]
    Y = np.array(Y)[filtered_idx]
    Z = np.array(Z)[filtered_idx]
    
    No_of_cells = X.shape[0]
    for j in range(0,len(region_list)):
        temp_no = 0
        temp_region = locals()[region_list[j]]
        for i in range(0,No_of_cells):
            if temp_region[int(Z[i]),int(Y[i]),int(X[i])] == 255:
                temp_no = temp_no+1
        np_region_NeuronNumber[1,j] = np_region_NeuronNumber[1,j]+temp_no
        
labels = region_list

x = np.arange(len(labels)) 
width = 0.3

fig, ax = plt.subplots(figsize = (5.5,4),dpi = 800)
rects1 = ax.bar(x - width/2, np_region_NeuronNumber[0,:], width, label='Freezing Cluster',color = 'dodgerblue')
rects2 = ax.bar(x + width/2, np_region_NeuronNumber[1,:], width, label='PC Cluster',color = 'crimson')
ax.set_ylabel('# of Neurons',fontsize = 15,labelpad = -1)
plt.ylim(0,1250)
plt.yticks(fontsize =15)
ax.set_xticks(x)
ax.set_xticklabels(labels,fontsize = 15)
plt.xlabel('Region',fontsize=14)
ax.legend(fontsize = 12,loc = 2)
ax.set_xticklabels(['Pretectum','Tectum','Cerebellum','Hindbrain'],rotation = 0,fontsize = 12)

def autolabel(rects):
    for rect in rects:
        height = rect.get_height()
        ax.annotate('{}'.format(int(height)),
                    xy=(rect.get_x() + rect.get_width() / 2, height),
                    xytext=(0, 3),  # 3 points vertical offset
                    textcoords="offset points",
                    ha='center', va='bottom')


autolabel(rects1)
autolabel(rects2)

plt.show()

In [ ]:
df_filtered_reg_clusters

In [ ]:
region_path_list

In [ ]:
region_path_list = glob.glob('C:\\Users\\yxton\\Documents\\Code\\2p\\freezing_project\\clustering_files\\mapzebrain_brain_regions\\*tif')
region_path_list = [region_path_list[0]]+[region_path_list[2]]+region_path_list[4:6]

In [ ]:
region_path_list

In [ ]:
np_fraction_cluster = np.zeros((len(cluster_idx),len(region_list)))
for temp_id,np_x in zip(cluster_idx,range(0,np_fraction_cluster.shape[0])):
    temp_id = int(temp_id)
    X = df_filtered_reg_clusters[df_filtered_reg_clusters['cluster']==temp_id].x/0.994
    Y = df_filtered_reg_clusters[df_filtered_reg_clusters['cluster']==temp_id].y/0.994
    Z = df_filtered_reg_clusters[df_filtered_reg_clusters['cluster']==temp_id].z
    
    filtered_idx = []
    for x,y,z,i in zip(X,Y,Z,range(0,X.shape[0])):
        if mask_xy[int(y),int(x)] == 255 and mask_xz[int(z),int(y)] == 255 and mask_yz[int(z),int(x)] == 255:
            filtered_idx.append(i)
            
    X = np.array(X)[filtered_idx]
    Y = np.array(Y)[filtered_idx]
    Z = np.array(Z)[filtered_idx]
    
    No_of_cells = X.shape[0]
    if No_of_cells>10:
        for j in range(0,np_fraction_cluster.shape[1]):
            temp_no = 0
            temp_region = locals()[region_list[j]]
            for i in range(0,No_of_cells):
                if temp_region[int(Z[i]),int(Y[i]),int(X[i])] == 255:
                    temp_no = temp_no+1
            np_fraction_cluster[np_x,j] = temp_no

np_idx_large_cluster = np.sum(np_fraction_cluster,axis = 1)!=0
np_fraction_large_cluster = np_fraction_cluster[np_idx_large_cluster]
cluster_idx_large = cluster_idx[np_idx_large_cluster]
No_cluster_idx_large = cluster_idx_large.shape[0]


np_fraction_large_cluster_dividecluster = np_fraction_large_cluster.copy()
np_fraction_large_cluster_divideregion = np_fraction_large_cluster.copy()

for x in range(0,np_fraction_large_cluster.shape[0]):
    np_fraction_large_cluster_dividecluster[x,:] = np_fraction_large_cluster_dividecluster[x,:]/np.sum(np_fraction_large_cluster_dividecluster[x,:])

for y in range(0,np_fraction_large_cluster.shape[1]):
    np_fraction_large_cluster_divideregion[:,y] = np_fraction_large_cluster_divideregion[:,y]/np.sum(np_fraction_large_cluster_divideregion[:,y])

            



fig,ax_list = plt.subplots(No_cluster_idx_large,1,figsize = (6,14),sharex =True,sharey =True,dpi=800)
color_list = plt.cm.viridis(np.linspace(0,1,No_cluster_idx_large))
for temp_id,i in zip(cluster_idx_large,range(0,No_cluster_idx_large)):
    ax_list[i].bar(region_list,np_fraction_large_cluster_dividecluster[i,:],color = color_list[i])
    plt.ylim(0,0.99)
#     plt.xticks([0,0.4,0.8])
    ax_list[i].set_xticklabels(region_list, rotation = 40,fontsize = 14,horizontalalignment = 'right')
    ax_list[i].set_ylabel('Cluster'+str(int(temp_id)),rotation = 0,loc= 'bottom',fontsize = 14,labelpad = 70)
    ax_list[i].set_yticklabels([0,0.5],fontsize = 8)
    
plt.suptitle('Anatomical location of clusters',fontsize =15,y = 0.9)
plt.show()


fig,ax_list = plt.subplots(No_cluster_idx_large,1,figsize = (6,14),sharex =True,sharey =True,dpi=800)
color_list = plt.cm.viridis(np.linspace(0,1,No_cluster_idx_large))
for temp_id,i in zip(cluster_idx_large,range(0,No_cluster_idx_large)):
    ax_list[i].bar(region_list,np_fraction_large_cluster_divideregion[i,:],color = color_list[i])
    plt.ylim(0,0.55)
    ax_list[i].set_xticklabels(region_list, rotation = 40,fontsize = 14,horizontalalignment = 'right')
    ax_list[i].set_ylabel('Cluster'+str(int(temp_id)),rotation = 0,loc= 'bottom',fontsize = 14,labelpad = 70)
    ax_list[i].set_yticklabels([0,0.5],fontsize = 8)
plt.suptitle('Fraction of imaged cells within each anatomical region',fontsize =15,y = 0.9)   
plt.show()

In [ ]:
# example neuron plotting
fig, ax = plt.subplots(1, 1, figsize=(15, 3),dpi = 800)

ax.set_ylabel('DeltaF/F', fontsize=20,labelpad = 4)
plt.ylim(-0.5, 3)
plt.xlim(0, 39*6)
plt.axvline(39, color='grey',alpha = 0.3)
plt.axvline(39*2, color='grey',alpha = 0.3)
plt.axvline(39*3, color='grey',alpha = 0.3)
plt.axvline(39*4, color='grey',alpha = 0.3)
plt.axvline(39*5, color='grey',alpha = 0.3)

ax.plot(np_trace_filtered[26,:],color = 'c',linewidth=4)
plt.fill_between(np.arange(27+39,30+39),3,-0.5,linewidth = 2,color = 'violet',alpha = 0.2)
plt.fill_between(np.arange(27+39*2,30+39*2),3,-0.5,linewidth = 2,color = 'violet',alpha = 0.2)
plt.fill_between(np.arange(27+39*3,30+39*3),3,-0.5,linewidth = 2,color = 'violet',alpha = 0.2)
plt.fill_between(np.arange(27+39*4,30+39*4),3,-0.5,linewidth = 2,color = 'violet',alpha = 0.2)
plt.fill_between(np.arange(27+39*5,30+39*5),3,-0.5,linewidth = 2,color = 'violet',alpha = 0.2)
plt.xticks([19,58,97,136,175,214])
plt.yticks([])
# plt.yticks(fontsize = 15)
ax.set_xticklabels(['No Stim', '4°', '15°', 'Luminance', 'Freezing (motor)', 'Prey (motor)'],fontsize = 18)
ax.tick_params(axis='x', which='both', length=0)
plt.show()

In [ ]:
clusters

In [ ]:
# the percentage of each fish in each cluster

df_percentage_cluster = pd.DataFrame(columns = ['cluster']+df_id_list)

for i in cluster_idx:
    i = int(i)
    locals()['cluster_'+str(i)+'_roi_idx'] = list(np.where(clusters == i)[0])
    
    if len(locals()['cluster_'+str(i)+'_roi_idx'])>10:
        temp_df_cluster = df_VMV_filtered.iloc[locals()['cluster_'+str(i)+'_roi_idx'],:]
        
        df_percentage_cluster_x = df_percentage_cluster.shape[0]
        df_percentage_cluster.loc[df_percentage_cluster_x,'cluster'] = i
        
        for main_dir,fish,df_id in zip(main_dir_list,fish_index_list,df_id_list):
            temp_fish_count =  np.where(np.logical_and(temp_df_cluster['Fish']==fish,temp_df_cluster['Date']==int(main_dir[-8:])))[0].shape[0]
            temp_ratio = temp_fish_count/temp_df_cluster.shape[0]
            df_percentage_cluster.loc[df_percentage_cluster_x,df_id] = temp_ratio
        
fig,ax = plt.subplots(1,1,figsize = (15,8),dpi=800)

bar_x = [str(i) for i in df_percentage_cluster['cluster']]
bottom = np.zeros(df_percentage_cluster.shape[0])
color_list = sns.color_palette("tab10")[:len(fish_index_list)]
for fish_i,color in zip(range(1,len(fish_index_list)+1),color_list):
    
    temp_ratio_list = df_percentage_cluster.iloc[:,fish_i]
    plt.bar(bar_x,temp_ratio_list,bottom=bottom,color = color)
    bottom = bottom+temp_ratio_list
    

plt.legend(df_id_list,bbox_to_anchor=(1.02, 1.0),fontsize = 15)
plt.ylim(0,1.05)
plt.xticks(fontsize = 15)
plt.xlabel('Cluster ID',fontsize = 20)
plt.yticks([0.0,0.2,0.4,0.6,0.8,1.0],['0%','20%','40%','60%','80%','100%'],fontsize =15)
plt.ylabel('Percentage',fontsize = 20)
plt.show()

In [ ]:
df_percentage_cluster

In [ ]:
locals()['cluster_'+str(i)+'_roi_idx']

In [ ]:
df_VMV_filtered

In [ ]:
df_percentage_cluster

# visualization of neurons in the atlas

In [ ]:
atlas_path = 'C:\\Users\\yxton\\Documents\\Code\\registration_code_HSA\\T_AVG_H2BGCaMP.tif'
atlas = tifffile.imread(atlas_path)

In [ ]:
mask_xy_path = 'C:\\Users\\yxton\\Documents\\Code\\2p\\freezing_project\\clustering_files\\Mask_xy.tif'
mask_xy = tifffile.imread(mask_xy_path)
mask_xz_path = 'C:\\Users\\yxton\\Documents\\Code\\2p\\freezing_project\\clustering_files\\Mask_xz.tif'
mask_xz = tifffile.imread(mask_xz_path)
mask_yz_path = 'C:\\Users\\yxton\\Documents\\Code\\2p\\freezing_project\\clustering_files\\Mask_yz.tif'
mask_yz = tifffile.imread(mask_yz_path)

In [ ]:
mask_xz.shape

In [ ]:
temp_cluster_id = 10
X = df_filtered_reg_clusters[df_filtered_reg_clusters['cluster']==temp_cluster_id].x/0.994
Y = df_filtered_reg_clusters[df_filtered_reg_clusters['cluster']==temp_cluster_id].y/0.994
Z = df_filtered_reg_clusters[df_filtered_reg_clusters['cluster']==temp_cluster_id].z

filtered_idx = []
for x,y,z,i in zip(X,Y,Z,range(0,X.shape[0])):
    if mask_xy[int(y),int(x)] == 255 and mask_xz[int(z),int(y)] == 255 and mask_yz[int(z),int(x)] == 255:
        filtered_idx.append(i)


fig,(ax1) = plt.subplots(1, 1,figsize = (5.97,9.74))
xy = np.mean(atlas,axis = 0)
yz = np.mean(atlas,axis = 1)
xz = np.mean(atlas,axis = 2)
ax = sns.heatmap(xy,ax = ax1,cmap = 'gray',cbar = False,vmin = 0,vmax = 30)

plt.scatter(np.array(X)[filtered_idx],np.array(Y)[filtered_idx],color = 'red',s = 1)
# ax = sns.heatmap(yz,ax = ax2,square = True,cmap = 'gray',cbar = False)

# ax = sns.heatmap(xz,ax = ax3,square = True,cmap = 'gray',cbar = False)
plt.axis('off')
plt.show()

fig,(ax1) = plt.subplots(1, 1,figsize = (9.74,3.59))
xy = np.mean(atlas,axis = 0)
yz = np.mean(atlas,axis = 1)
xz = np.mean(atlas,axis = 2)
ax = sns.heatmap(xz,ax = ax1,cmap = 'gray',cbar = False,vmin = 0,vmax = 30)

plt.scatter(np.array(Y)[filtered_idx],np.array(Z)[filtered_idx],color = 'red',s = 1)
# ax = sns.heatmap(yz,ax = ax2,square = True,cmap = 'gray',cbar = False)

# ax = sns.heatmap(xz,ax = ax3,square = True,cmap = 'gray',cbar = False)
plt.axis('off')
plt.show()

fig,(ax1) = plt.subplots(1, 1,figsize = (5.94,3.59))
xy = np.mean(atlas,axis = 0)
yz = np.mean(atlas,axis = 1)
xz = np.mean(atlas,axis = 2)
ax = sns.heatmap(yz,ax = ax1,cmap = 'gray',cbar = False,vmin = 0,vmax = 30)

plt.scatter(np.array(X)[filtered_idx],np.array(Z)[filtered_idx],color = 'red',s = 1)
# ax = sns.heatmap(yz,ax = ax2,square = True,cmap = 'gray',cbar = False)

# ax = sns.heatmap(xz,ax = ax3,square = True,cmap = 'gray',cbar = False)
plt.axis('off')
plt.show()

In [ ]:
temp_cluster_id = 100
print('Cluster'+str(temp_cluster_id))
colorcode = 'cyan'
size = 2


X = df_filtered_reg_clusters[df_filtered_reg_clusters['cluster']==temp_cluster_id].x/0.994
Y = df_filtered_reg_clusters[df_filtered_reg_clusters['cluster']==temp_cluster_id].y/0.994
Z = df_filtered_reg_clusters[df_filtered_reg_clusters['cluster']==temp_cluster_id].z

filtered_idx = []
for x,y,z,i in zip(X,Y,Z,range(0,X.shape[0])):
    if mask_xy[int(y),int(x)] == 255 and mask_xz[int(z),int(y)] == 255 and mask_yz[int(z),int(x)] == 255:
        filtered_idx.append(i)


fig,(ax1) = plt.subplots(1, 1,figsize = (5.97,9.74))
xy = np.mean(atlas,axis = 0)
yz = np.mean(atlas,axis = 1)
xz = np.mean(atlas,axis = 2)
ax = sns.heatmap(xy,ax = ax1,cmap = 'gray',cbar = False,vmin = 0,vmax = 30)

plt.scatter(np.array(X)[filtered_idx],np.array(Y)[filtered_idx],color = colorcode,s = size)
# ax = sns.heatmap(yz,ax = ax2,square = True,cmap = 'gray',cbar = False)

# ax = sns.heatmap(xz,ax = ax3,square = True,cmap = 'gray',cbar = False)
plt.axis('off')
plt.show()

fig,(ax1) = plt.subplots(1, 1,figsize = (9.74,3.59))
xy = np.mean(atlas,axis = 0)
yz = np.mean(atlas,axis = 1)
xz = np.mean(atlas,axis = 2)
ax = sns.heatmap(xz,ax = ax1,cmap = 'gray',cbar = False,vmin = 0,vmax = 30)

plt.scatter(np.array(Y)[filtered_idx],np.array(Z)[filtered_idx],color = colorcode,s = size)
# ax = sns.heatmap(yz,ax = ax2,square = True,cmap = 'gray',cbar = False)

# ax = sns.heatmap(xz,ax = ax3,square = True,cmap = 'gray',cbar = False)
plt.axis('off')
plt.show()

fig,(ax1) = plt.subplots(1, 1,figsize = (5.97,3.59))
xy = np.mean(atlas,axis = 0)
yz = np.mean(atlas,axis = 1)
xz = np.mean(atlas,axis = 2)
ax = sns.heatmap(yz,ax = ax1,cmap = 'gray',cbar = False,vmin = 0,vmax = 30)

plt.scatter(np.array(X)[filtered_idx],np.array(Z)[filtered_idx],color = colorcode,s = size)
# ax = sns.heatmap(yz,ax = ax2,square = True,cmap = 'gray',cbar = False)

# ax = sns.heatmap(xz,ax = ax3,square = True,cmap = 'gray',cbar = False)
plt.axis('off')
plt.show()

In [ ]:
temp_cluster_id = [75]
print('Cluster'+str(temp_cluster_id))
colorcode = 'cyan'
size = 2
vmax = 50

X = pd.Series()
Y = pd.Series()
Z = pd.Series()

for temp_id in temp_cluster_id:
    X = X.append(df_filtered_reg_clusters[df_filtered_reg_clusters['cluster']==temp_id].x/0.994)
    Y = Y.append(df_filtered_reg_clusters[df_filtered_reg_clusters['cluster']==temp_id].y/0.994)
    Z = Z.append(df_filtered_reg_clusters[df_filtered_reg_clusters['cluster']==temp_id].z)


filtered_idx = []
for x,y,z,i in zip(X,Y,Z,range(0,X.shape[0])):
    if mask_xy[int(y),int(x)] == 255 and mask_xz[int(z),int(y)] == 255 and mask_yz[int(z),int(x)] == 255:
        filtered_idx.append(i)


fig,(ax1) = plt.subplots(1, 1,figsize = (5.97,9.74))
xy = np.mean(atlas,axis = 0)
yz = np.mean(atlas,axis = 1)
xz = np.mean(atlas,axis = 2)
ax = sns.heatmap(xy,ax = ax1,cmap = 'gray',cbar = False,vmin = 0,vmax = vmax)

plt.scatter(np.array(X)[filtered_idx],np.array(Y)[filtered_idx],color = colorcode,s = size)
# ax = sns.heatmap(yz,ax = ax2,square = True,cmap = 'gray',cbar = False)

# ax = sns.heatmap(xz,ax = ax3,square = True,cmap = 'gray',cbar = False)
plt.axis('off')
fig.tight_layout()
plt.show()

fig,(ax1) = plt.subplots(1, 1,figsize = (9.74,3.59))
xy = np.mean(atlas,axis = 0)
yz = np.mean(atlas,axis = 1)
xz = np.mean(atlas,axis = 2)
ax = sns.heatmap(xz,ax = ax1,cmap = 'gray',cbar = False,vmin = 0,vmax = vmax)

plt.scatter(np.array(Y)[filtered_idx],np.array(Z)[filtered_idx],color = colorcode,s = size)
# ax = sns.heatmap(yz,ax = ax2,square = True,cmap = 'gray',cbar = False)

# ax = sns.heatmap(xz,ax = ax3,square = True,cmap = 'gray',cbar = False)
plt.axis('off')
fig.tight_layout()
plt.show()

fig,(ax1) = plt.subplots(1, 1,figsize = (5.97,3.59))
xy = np.mean(atlas,axis = 0)
yz = np.mean(atlas,axis = 1)
xz = np.mean(atlas,axis = 2)
ax = sns.heatmap(yz,ax = ax1,cmap = 'gray',cbar = False,vmin = 0,vmax = vmax)

plt.scatter(np.array(X)[filtered_idx],np.array(Z)[filtered_idx],color = colorcode,s = size)
# ax = sns.heatmap(yz,ax = ax2,square = True,cmap = 'gray',cbar = False)

# ax = sns.heatmap(xz,ax = ax3,square = True,cmap = 'gray',cbar = False)
plt.axis('off')
fig.tight_layout()
plt.show()

In [ ]:
temp_cluster_id = [28]
print('Cluster'+str(temp_cluster_id))
colorcode = 'red'
size = 2
vmax = 50

X = pd.Series()
Y = pd.Series()
Z = pd.Series()

for temp_id in temp_cluster_id:
    X = X.append(df_filtered_reg_clusters[df_filtered_reg_clusters['cluster']==temp_id].x/0.994)
    Y = Y.append(df_filtered_reg_clusters[df_filtered_reg_clusters['cluster']==temp_id].y/0.994)
    Z = Z.append(df_filtered_reg_clusters[df_filtered_reg_clusters['cluster']==temp_id].z)


filtered_idx = []
for x,y,z,i in zip(X,Y,Z,range(0,X.shape[0])):
    if mask_xy[int(y),int(x)] == 255 and mask_xz[int(z),int(y)] == 255 and mask_yz[int(z),int(x)] == 255:
        filtered_idx.append(i)


fig,(ax1) = plt.subplots(1, 1,figsize = (5.97,9.74))
xy = np.mean(atlas,axis = 0)
yz = np.mean(atlas,axis = 1)
xz = np.mean(atlas,axis = 2)
ax = sns.heatmap(xy,ax = ax1,cmap = 'gray',cbar = False,vmin = 0,vmax = vmax)

plt.scatter(np.array(X)[filtered_idx],np.array(Y)[filtered_idx],color = colorcode,s = size)
# ax = sns.heatmap(yz,ax = ax2,square = True,cmap = 'gray',cbar = False)

# ax = sns.heatmap(xz,ax = ax3,square = True,cmap = 'gray',cbar = False)
plt.axis('off')
fig.tight_layout()
plt.show()

fig,(ax1) = plt.subplots(1, 1,figsize = (9.74,3.59))
xy = np.mean(atlas,axis = 0)
yz = np.mean(atlas,axis = 1)
xz = np.mean(atlas,axis = 2)
ax = sns.heatmap(xz,ax = ax1,cmap = 'gray',cbar = False,vmin = 0,vmax = vmax)

plt.scatter(np.array(Y)[filtered_idx],np.array(Z)[filtered_idx],color = colorcode,s = size)
# ax = sns.heatmap(yz,ax = ax2,square = True,cmap = 'gray',cbar = False)

# ax = sns.heatmap(xz,ax = ax3,square = True,cmap = 'gray',cbar = False)
plt.axis('off')
fig.tight_layout()
plt.show()

fig,(ax1) = plt.subplots(1, 1,figsize = (5.97,3.59))
xy = np.mean(atlas,axis = 0)
yz = np.mean(atlas,axis = 1)
xz = np.mean(atlas,axis = 2)
ax = sns.heatmap(yz,ax = ax1,cmap = 'gray',cbar = False,vmin = 0,vmax = vmax)

plt.scatter(np.array(X)[filtered_idx],np.array(Z)[filtered_idx],color = colorcode,s = size)
# ax = sns.heatmap(yz,ax = ax2,square = True,cmap = 'gray',cbar = False)

# ax = sns.heatmap(xz,ax = ax3,square = True,cmap = 'gray',cbar = False)
plt.axis('off')
fig.tight_layout()
plt.show()

In [ ]:
temp_cluster_id = [70]
print('Cluster'+str(temp_cluster_id))
colorcode = 'yellow'
size = 2
vmax = 50

X = pd.Series()
Y = pd.Series()
Z = pd.Series()

for temp_id in temp_cluster_id:
    X = X.append(df_filtered_reg_clusters[df_filtered_reg_clusters['cluster']==temp_id].x/0.994)
    Y = Y.append(df_filtered_reg_clusters[df_filtered_reg_clusters['cluster']==temp_id].y/0.994)
    Z = Z.append(df_filtered_reg_clusters[df_filtered_reg_clusters['cluster']==temp_id].z)


filtered_idx = []
for x,y,z,i in zip(X,Y,Z,range(0,X.shape[0])):
    if mask_xy[int(y),int(x)] == 255 and mask_xz[int(z),int(y)] == 255 and mask_yz[int(z),int(x)] == 255:
        filtered_idx.append(i)


fig,(ax1) = plt.subplots(1, 1,figsize = (5.97,9.74))
xy = np.mean(atlas,axis = 0)
yz = np.mean(atlas,axis = 1)
xz = np.mean(atlas,axis = 2)
ax = sns.heatmap(xy,ax = ax1,cmap = 'gray',cbar = False,vmin = 0,vmax = vmax)

plt.scatter(np.array(X)[filtered_idx],np.array(Y)[filtered_idx],color = colorcode,s = size)
# ax = sns.heatmap(yz,ax = ax2,square = True,cmap = 'gray',cbar = False)

# ax = sns.heatmap(xz,ax = ax3,square = True,cmap = 'gray',cbar = False)
plt.axis('off')
fig.tight_layout()
plt.show()

fig,(ax1) = plt.subplots(1, 1,figsize = (9.74,3.59))
xy = np.mean(atlas,axis = 0)
yz = np.mean(atlas,axis = 1)
xz = np.mean(atlas,axis = 2)
ax = sns.heatmap(xz,ax = ax1,cmap = 'gray',cbar = False,vmin = 0,vmax = vmax)

plt.scatter(np.array(Y)[filtered_idx],np.array(Z)[filtered_idx],color = colorcode,s = size)
# ax = sns.heatmap(yz,ax = ax2,square = True,cmap = 'gray',cbar = False)

# ax = sns.heatmap(xz,ax = ax3,square = True,cmap = 'gray',cbar = False)
plt.axis('off')
fig.tight_layout()
plt.show()

fig,(ax1) = plt.subplots(1, 1,figsize = (5.97,3.59))
xy = np.mean(atlas,axis = 0)
yz = np.mean(atlas,axis = 1)
xz = np.mean(atlas,axis = 2)
ax = sns.heatmap(yz,ax = ax1,cmap = 'gray',cbar = False,vmin = 0,vmax = vmax)

plt.scatter(np.array(X)[filtered_idx],np.array(Z)[filtered_idx],color = colorcode,s = size)
# ax = sns.heatmap(yz,ax = ax2,square = True,cmap = 'gray',cbar = False)

# ax = sns.heatmap(xz,ax = ax3,square = True,cmap = 'gray',cbar = False)
plt.axis('off')
fig.tight_layout()
plt.show()

In [ ]:
temp_cluster_id = [31,32]
print('Cluster'+str(temp_cluster_id))
colorcode = 'orange'
size = 2
vmax = 50

X = pd.Series()
Y = pd.Series()
Z = pd.Series()

for temp_id in temp_cluster_id:
    X = X.append(df_filtered_reg_clusters[df_filtered_reg_clusters['cluster']==temp_id].x/0.994)
    Y = Y.append(df_filtered_reg_clusters[df_filtered_reg_clusters['cluster']==temp_id].y/0.994)
    Z = Z.append(df_filtered_reg_clusters[df_filtered_reg_clusters['cluster']==temp_id].z)


filtered_idx = []
for x,y,z,i in zip(X,Y,Z,range(0,X.shape[0])):
    if mask_xy[int(y),int(x)] == 255 and mask_xz[int(z),int(y)] == 255 and mask_yz[int(z),int(x)] == 255:
        filtered_idx.append(i)


fig,(ax1) = plt.subplots(1, 1,figsize = (5.97,9.74))
xy = np.mean(atlas,axis = 0)
yz = np.mean(atlas,axis = 1)
xz = np.mean(atlas,axis = 2)
ax = sns.heatmap(xy,ax = ax1,cmap = 'gray',cbar = False,vmin = 0,vmax = vmax)

plt.scatter(np.array(X)[filtered_idx],np.array(Y)[filtered_idx],color = colorcode,s = size)
# ax = sns.heatmap(yz,ax = ax2,square = True,cmap = 'gray',cbar = False)

# ax = sns.heatmap(xz,ax = ax3,square = True,cmap = 'gray',cbar = False)
plt.axis('off')
fig.tight_layout()
plt.show()

fig,(ax1) = plt.subplots(1, 1,figsize = (9.74,3.59))
xy = np.mean(atlas,axis = 0)
yz = np.mean(atlas,axis = 1)
xz = np.mean(atlas,axis = 2)
ax = sns.heatmap(xz,ax = ax1,cmap = 'gray',cbar = False,vmin = 0,vmax = vmax)

plt.scatter(np.array(Y)[filtered_idx],np.array(Z)[filtered_idx],color = colorcode,s = size)
# ax = sns.heatmap(yz,ax = ax2,square = True,cmap = 'gray',cbar = False)

# ax = sns.heatmap(xz,ax = ax3,square = True,cmap = 'gray',cbar = False)
plt.axis('off')
fig.tight_layout()
plt.show()

fig,(ax1) = plt.subplots(1, 1,figsize = (5.97,3.59))
xy = np.mean(atlas,axis = 0)
yz = np.mean(atlas,axis = 1)
xz = np.mean(atlas,axis = 2)
ax = sns.heatmap(yz,ax = ax1,cmap = 'gray',cbar = False,vmin = 0,vmax = vmax)

plt.scatter(np.array(X)[filtered_idx],np.array(Z)[filtered_idx],color = colorcode,s = size)
# ax = sns.heatmap(yz,ax = ax2,square = True,cmap = 'gray',cbar = False)

# ax = sns.heatmap(xz,ax = ax3,square = True,cmap = 'gray',cbar = False)
plt.axis('off')
fig.tight_layout()
plt.show()

In [ ]:
temp_cluster_id = [67]
print('Cluster'+str(temp_cluster_id))
colorcode = 'lime'
size = 2
vmax = 50

X = pd.Series()
Y = pd.Series()
Z = pd.Series()

for temp_id in temp_cluster_id:
    X = X.append(df_filtered_reg_clusters[df_filtered_reg_clusters['cluster']==temp_id].x/0.994)
    Y = Y.append(df_filtered_reg_clusters[df_filtered_reg_clusters['cluster']==temp_id].y/0.994)
    Z = Z.append(df_filtered_reg_clusters[df_filtered_reg_clusters['cluster']==temp_id].z)


filtered_idx = []
for x,y,z,i in zip(X,Y,Z,range(0,X.shape[0])):
    if mask_xy[int(y),int(x)] == 255 and mask_xz[int(z),int(y)] == 255 and mask_yz[int(z),int(x)] == 255:
        filtered_idx.append(i)


fig,(ax1) = plt.subplots(1, 1,figsize = (5.97,9.74))
xy = np.mean(atlas,axis = 0)
yz = np.mean(atlas,axis = 1)
xz = np.mean(atlas,axis = 2)
ax = sns.heatmap(xy,ax = ax1,cmap = 'gray',cbar = False,vmin = 0,vmax = vmax)

plt.scatter(np.array(X)[filtered_idx],np.array(Y)[filtered_idx],color = colorcode,s = size)
# ax = sns.heatmap(yz,ax = ax2,square = True,cmap = 'gray',cbar = False)

# ax = sns.heatmap(xz,ax = ax3,square = True,cmap = 'gray',cbar = False)
plt.axis('off')
fig.tight_layout()
plt.show()

fig,(ax1) = plt.subplots(1, 1,figsize = (9.74,3.59))
xy = np.mean(atlas,axis = 0)
yz = np.mean(atlas,axis = 1)
xz = np.mean(atlas,axis = 2)
ax = sns.heatmap(xz,ax = ax1,cmap = 'gray',cbar = False,vmin = 0,vmax = vmax)

plt.scatter(np.array(Y)[filtered_idx],np.array(Z)[filtered_idx],color = colorcode,s = size)
# ax = sns.heatmap(yz,ax = ax2,square = True,cmap = 'gray',cbar = False)

# ax = sns.heatmap(xz,ax = ax3,square = True,cmap = 'gray',cbar = False)
plt.axis('off')
fig.tight_layout()
plt.show()

fig,(ax1) = plt.subplots(1, 1,figsize = (5.97,3.59))
xy = np.mean(atlas,axis = 0)
yz = np.mean(atlas,axis = 1)
xz = np.mean(atlas,axis = 2)
ax = sns.heatmap(yz,ax = ax1,cmap = 'gray',cbar = False,vmin = 0,vmax = vmax)

plt.scatter(np.array(X)[filtered_idx],np.array(Z)[filtered_idx],color = colorcode,s = size)
# ax = sns.heatmap(yz,ax = ax2,square = True,cmap = 'gray',cbar = False)

# ax = sns.heatmap(xz,ax = ax3,square = True,cmap = 'gray',cbar = False)
plt.axis('off')
fig.tight_layout()
plt.show()

In [ ]:
temp_cluster_id = [79]
print('Cluster'+str(temp_cluster_id))
colorcode = 'magenta'
size = 2
vmax = 50

X = pd.Series()
Y = pd.Series()
Z = pd.Series()

for temp_id in temp_cluster_id:
    X = X.append(df_filtered_reg_clusters[df_filtered_reg_clusters['cluster']==temp_id].x/0.994)
    Y = Y.append(df_filtered_reg_clusters[df_filtered_reg_clusters['cluster']==temp_id].y/0.994)
    Z = Z.append(df_filtered_reg_clusters[df_filtered_reg_clusters['cluster']==temp_id].z)


filtered_idx = []
for x,y,z,i in zip(X,Y,Z,range(0,X.shape[0])):
    if mask_xy[int(y),int(x)] == 255 and mask_xz[int(z),int(y)] == 255 and mask_yz[int(z),int(x)] == 255:
        filtered_idx.append(i)


fig,(ax1) = plt.subplots(1, 1,figsize = (5.97,9.74))
xy = np.mean(atlas,axis = 0)
yz = np.mean(atlas,axis = 1)
xz = np.mean(atlas,axis = 2)
ax = sns.heatmap(xy,ax = ax1,cmap = 'gray',cbar = False,vmin = 0,vmax = vmax)

plt.scatter(np.array(X)[filtered_idx],np.array(Y)[filtered_idx],color = colorcode,s = size)
# ax = sns.heatmap(yz,ax = ax2,square = True,cmap = 'gray',cbar = False)

# ax = sns.heatmap(xz,ax = ax3,square = True,cmap = 'gray',cbar = False)
plt.axis('off')
fig.tight_layout()
plt.show()

fig,(ax1) = plt.subplots(1, 1,figsize = (9.74,3.59))
xy = np.mean(atlas,axis = 0)
yz = np.mean(atlas,axis = 1)
xz = np.mean(atlas,axis = 2)
ax = sns.heatmap(xz,ax = ax1,cmap = 'gray',cbar = False,vmin = 0,vmax = vmax)

plt.scatter(np.array(Y)[filtered_idx],np.array(Z)[filtered_idx],color = colorcode,s = size)
# ax = sns.heatmap(yz,ax = ax2,square = True,cmap = 'gray',cbar = False)

# ax = sns.heatmap(xz,ax = ax3,square = True,cmap = 'gray',cbar = False)
plt.axis('off')
fig.tight_layout()
plt.show()

fig,(ax1) = plt.subplots(1, 1,figsize = (5.97,3.59))
xy = np.mean(atlas,axis = 0)
yz = np.mean(atlas,axis = 1)
xz = np.mean(atlas,axis = 2)
ax = sns.heatmap(yz,ax = ax1,cmap = 'gray',cbar = False,vmin = 0,vmax = vmax)

plt.scatter(np.array(X)[filtered_idx],np.array(Z)[filtered_idx],color = colorcode,s = size)
# ax = sns.heatmap(yz,ax = ax2,square = True,cmap = 'gray',cbar = False)

# ax = sns.heatmap(xz,ax = ax3,square = True,cmap = 'gray',cbar = False)
plt.axis('off')
fig.tight_layout()
plt.show()

In [ ]:
temp_cluster_id = [87]
print('Cluster'+str(temp_cluster_id))
colorcode = 'gold'
size = 2
vmax = 50

X = pd.Series()
Y = pd.Series()
Z = pd.Series()

for temp_id in temp_cluster_id:
    X = X.append(df_filtered_reg_clusters[df_filtered_reg_clusters['cluster']==temp_id].x/0.994)
    Y = Y.append(df_filtered_reg_clusters[df_filtered_reg_clusters['cluster']==temp_id].y/0.994)
    Z = Z.append(df_filtered_reg_clusters[df_filtered_reg_clusters['cluster']==temp_id].z)


filtered_idx = []
for x,y,z,i in zip(X,Y,Z,range(0,X.shape[0])):
    if mask_xy[int(y),int(x)] == 255 and mask_xz[int(z),int(y)] == 255 and mask_yz[int(z),int(x)] == 255:
        filtered_idx.append(i)


fig,(ax1) = plt.subplots(1, 1,figsize = (5.97,9.74))
xy = np.mean(atlas,axis = 0)
yz = np.mean(atlas,axis = 1)
xz = np.mean(atlas,axis = 2)
ax = sns.heatmap(xy,ax = ax1,cmap = 'gray',cbar = False,vmin = 0,vmax = vmax)

plt.scatter(np.array(X)[filtered_idx],np.array(Y)[filtered_idx],color = colorcode,s = size)
# ax = sns.heatmap(yz,ax = ax2,square = True,cmap = 'gray',cbar = False)

# ax = sns.heatmap(xz,ax = ax3,square = True,cmap = 'gray',cbar = False)
plt.axis('off')
fig.tight_layout()
plt.show()

fig,(ax1) = plt.subplots(1, 1,figsize = (9.74,3.59))
xy = np.mean(atlas,axis = 0)
yz = np.mean(atlas,axis = 1)
xz = np.mean(atlas,axis = 2)
ax = sns.heatmap(xz,ax = ax1,cmap = 'gray',cbar = False,vmin = 0,vmax = vmax)

plt.scatter(np.array(Y)[filtered_idx],np.array(Z)[filtered_idx],color = colorcode,s = size)
# ax = sns.heatmap(yz,ax = ax2,square = True,cmap = 'gray',cbar = False)

# ax = sns.heatmap(xz,ax = ax3,square = True,cmap = 'gray',cbar = False)
plt.axis('off')
fig.tight_layout()
plt.show()

fig,(ax1) = plt.subplots(1, 1,figsize = (5.97,3.59))
xy = np.mean(atlas,axis = 0)
yz = np.mean(atlas,axis = 1)
xz = np.mean(atlas,axis = 2)
ax = sns.heatmap(yz,ax = ax1,cmap = 'gray',cbar = False,vmin = 0,vmax = vmax)

plt.scatter(np.array(X)[filtered_idx],np.array(Z)[filtered_idx],color = colorcode,s = size)
# ax = sns.heatmap(yz,ax = ax2,square = True,cmap = 'gray',cbar = False)

# ax = sns.heatmap(xz,ax = ax3,square = True,cmap = 'gray',cbar = False)
plt.axis('off')
fig.tight_layout()
plt.show()

In [ ]:
temp_cluster_id = [88]
print('Cluster'+str(temp_cluster_id))
colorcode = 'gold'
size = 2
vmax = 50

X = pd.Series()
Y = pd.Series()
Z = pd.Series()

for temp_id in temp_cluster_id:
    X = X.append(df_filtered_reg_clusters[df_filtered_reg_clusters['cluster']==temp_id].x/0.994)
    Y = Y.append(df_filtered_reg_clusters[df_filtered_reg_clusters['cluster']==temp_id].y/0.994)
    Z = Z.append(df_filtered_reg_clusters[df_filtered_reg_clusters['cluster']==temp_id].z)


filtered_idx = []
for x,y,z,i in zip(X,Y,Z,range(0,X.shape[0])):
    if mask_xy[int(y),int(x)] == 255 and mask_xz[int(z),int(y)] == 255 and mask_yz[int(z),int(x)] == 255:
        filtered_idx.append(i)


fig,(ax1) = plt.subplots(1, 1,figsize = (5.97,9.74))
xy = np.mean(atlas,axis = 0)
yz = np.mean(atlas,axis = 1)
xz = np.mean(atlas,axis = 2)
ax = sns.heatmap(xy,ax = ax1,cmap = 'gray',cbar = False,vmin = 0,vmax = vmax)

plt.scatter(np.array(X)[filtered_idx],np.array(Y)[filtered_idx],color = colorcode,s = size)
# ax = sns.heatmap(yz,ax = ax2,square = True,cmap = 'gray',cbar = False)

# ax = sns.heatmap(xz,ax = ax3,square = True,cmap = 'gray',cbar = False)
plt.axis('off')
fig.tight_layout()
plt.show()

fig,(ax1) = plt.subplots(1, 1,figsize = (9.74,3.59))
xy = np.mean(atlas,axis = 0)
yz = np.mean(atlas,axis = 1)
xz = np.mean(atlas,axis = 2)
ax = sns.heatmap(xz,ax = ax1,cmap = 'gray',cbar = False,vmin = 0,vmax = vmax)

plt.scatter(np.array(Y)[filtered_idx],np.array(Z)[filtered_idx],color = colorcode,s = size)
# ax = sns.heatmap(yz,ax = ax2,square = True,cmap = 'gray',cbar = False)

# ax = sns.heatmap(xz,ax = ax3,square = True,cmap = 'gray',cbar = False)
plt.axis('off')
fig.tight_layout()
plt.show()

fig,(ax1) = plt.subplots(1, 1,figsize = (5.97,3.59))
xy = np.mean(atlas,axis = 0)
yz = np.mean(atlas,axis = 1)
xz = np.mean(atlas,axis = 2)
ax = sns.heatmap(yz,ax = ax1,cmap = 'gray',cbar = False,vmin = 0,vmax = vmax)

plt.scatter(np.array(X)[filtered_idx],np.array(Z)[filtered_idx],color = colorcode,s = size)
# ax = sns.heatmap(yz,ax = ax2,square = True,cmap = 'gray',cbar = False)

# ax = sns.heatmap(xz,ax = ax3,square = True,cmap = 'gray',cbar = False)
plt.axis('off')
fig.tight_layout()
plt.show()

# anatomical quantifications

In [ ]:
# brain region


region_path_list = glob.glob('C:\\Users\\yxton\\Documents\\Code\\2p\\freezing_project\\clustering_files\\mapzebrain_brain_regions\\*tif')
region_list = []

for path in region_path_list:
    region = path.split('\\')[-1][:-4]
    locals()[region] = tifffile.imread(path)
    region_list.append(region)

In [ ]:
np_fraction_cluster = np.zeros((len(cluster_idx),len(region_list)))
for temp_id,np_x in zip(cluster_idx,range(0,np_fraction_cluster.shape[0])):
    temp_id = int(temp_id)
    X = df_filtered_reg_clusters[df_filtered_reg_clusters['cluster']==temp_id].x/0.994
    Y = df_filtered_reg_clusters[df_filtered_reg_clusters['cluster']==temp_id].y/0.994
    Z = df_filtered_reg_clusters[df_filtered_reg_clusters['cluster']==temp_id].z
    
    filtered_idx = []
    for x,y,z,i in zip(X,Y,Z,range(0,X.shape[0])):
        if mask_xy[int(y),int(x)] == 255 and mask_xz[int(z),int(y)] == 255 and mask_yz[int(z),int(x)] == 255:
            filtered_idx.append(i)
            
    X = np.array(X)[filtered_idx]
    Y = np.array(Y)[filtered_idx]
    Z = np.array(Z)[filtered_idx]
    
    No_of_cells = X.shape[0]
    if No_of_cells>10:
        for j in range(0,np_fraction_cluster.shape[1]):
            temp_no = 0
            temp_region = locals()[region_list[j]]
            for i in range(0,No_of_cells):
                if temp_region[int(Z[i]),int(Y[i]),int(X[i])] == 255:
                    temp_no = temp_no+1
            np_fraction_cluster[np_x,j] = temp_no

np_idx_large_cluster = np.sum(np_fraction_cluster,axis = 1)!=0
np_fraction_large_cluster = np_fraction_cluster[np_idx_large_cluster]
cluster_idx_large = cluster_idx[np_idx_large_cluster]
No_cluster_idx_large = cluster_idx_large.shape[0]


np_fraction_large_cluster_dividecluster = np_fraction_large_cluster.copy()
np_fraction_large_cluster_divideregion = np_fraction_large_cluster.copy()

for x in range(0,np_fraction_large_cluster.shape[0]):
    np_fraction_large_cluster_dividecluster[x,:] = np_fraction_large_cluster_dividecluster[x,:]/np.sum(np_fraction_large_cluster_dividecluster[x,:])

for y in range(0,np_fraction_large_cluster.shape[1]):
    np_fraction_large_cluster_divideregion[:,y] = np_fraction_large_cluster_divideregion[:,y]/np.sum(np_fraction_large_cluster_divideregion[:,y])

            



fig,ax_list = plt.subplots(No_cluster_idx_large,1,figsize = (6,14),sharex =True,sharey =True,dpi=800)
color_list = plt.cm.viridis(np.linspace(0,1,No_cluster_idx_large))
for temp_id,i in zip(cluster_idx_large,range(0,No_cluster_idx_large)):
    ax_list[i].bar(region_list,np_fraction_large_cluster_dividecluster[i,:],color = color_list[i])
    plt.ylim(0,0.99)
#     plt.xticks([0,0.4,0.8])
    ax_list[i].set_xticklabels(region_list, rotation = 40,fontsize = 14,horizontalalignment = 'right')
    ax_list[i].set_ylabel('Cluster'+str(int(temp_id)),rotation = 0,loc= 'bottom',fontsize = 14,labelpad = 70)
    ax_list[i].set_yticklabels([0,0.5],fontsize = 8)
    
plt.suptitle('Anatomical location of clusters',fontsize =15,y = 0.9)
plt.show()


fig,ax_list = plt.subplots(No_cluster_idx_large,1,figsize = (6,14),sharex =True,sharey =True,dpi=800)
color_list = plt.cm.viridis(np.linspace(0,1,No_cluster_idx_large))
for temp_id,i in zip(cluster_idx_large,range(0,No_cluster_idx_large)):
    ax_list[i].bar(region_list,np_fraction_large_cluster_divideregion[i,:],color = color_list[i])
    plt.ylim(0,0.55)
    ax_list[i].set_xticklabels(region_list, rotation = 40,fontsize = 14,horizontalalignment = 'right')
    ax_list[i].set_ylabel('Cluster'+str(int(temp_id)),rotation = 0,loc= 'bottom',fontsize = 14,labelpad = 70)
    ax_list[i].set_yticklabels([0,0.5],fontsize = 8)
plt.suptitle('Fraction of imaged cells within each anatomical region',fontsize =15,y = 0.9)   
plt.show()

In [ ]:
# the uniformness among fish
main_dir_list = ["F:\\LCr_UV_HSA_Lum_Low_1\\20230312"]*5+["F:\\LCr_UV_HSA_Lum_Low_2\\20230409"]*3+["F:\\LCr_UV_HSA_Lum_Low_2\\20230410"]*2
fish_index_list = ['F1_10_40','F2_12_20','F4_16_00','F5_18_00','F6_20_00','F2_13_00','F4_17_00','F5_19_00','F2_13_00','F4_16_00']

for main_dir,fish_index in zip(main_dir_list,fish_index_list):
    temp_date = main_dir.split('\\')[-1]
    temp_fish = fish_index[:2]
    df_filtered_reg_clusters_fish = df_filtered_reg_clusters[np.logical_and(df_filtered_reg_clusters['date'] == int(temp_date),df_filtered_reg_clusters['fish'] == temp_fish)]

    np_fraction_cluster = np.zeros((len(cluster_idx),len(region_list)))
    for temp_id,np_x in zip(cluster_idx,range(0,np_fraction_cluster.shape[0])):
        temp_id = int(temp_id)
        X = df_filtered_reg_clusters_fish[df_filtered_reg_clusters_fish['cluster']==temp_id].x/0.994
        Y = df_filtered_reg_clusters_fish[df_filtered_reg_clusters_fish['cluster']==temp_id].y/0.994
        Z = df_filtered_reg_clusters_fish[df_filtered_reg_clusters_fish['cluster']==temp_id].z

        filtered_idx = []
        for x,y,z,i in zip(X,Y,Z,range(0,X.shape[0])):
            if mask_xy[int(y),int(x)] == 255 and mask_xz[int(z),int(y)] == 255 and mask_yz[int(z),int(x)] == 255:
                filtered_idx.append(i)

        X = np.array(X)[filtered_idx]
        Y = np.array(Y)[filtered_idx]
        Z = np.array(Z)[filtered_idx]

        No_of_cells = X.shape[0]
        if No_of_cells>2:
            for j in range(0,np_fraction_cluster.shape[1]):
                temp_no = 0
                temp_region = locals()[region_list[j]]
                for i in range(0,No_of_cells):
                    if temp_region[int(Z[i]),int(Y[i]),int(X[i])] == 255:
                        temp_no = temp_no+1
                np_fraction_cluster[np_x,j] = temp_no

    np_idx_large_cluster = np.sum(np_fraction_cluster,axis = 1)!=0
    np_fraction_large_cluster = np_fraction_cluster[np_idx_large_cluster]
    cluster_idx_large = cluster_idx[np_idx_large_cluster]
    No_cluster_idx_large = cluster_idx_large.shape[0]


    np_fraction_large_cluster_dividecluster = np_fraction_large_cluster.copy()
    np_fraction_large_cluster_divideregion = np_fraction_large_cluster.copy()

    for x in range(0,np_fraction_large_cluster.shape[0]):
        np_fraction_large_cluster_dividecluster[x,:] = np_fraction_large_cluster_dividecluster[x,:]/np.sum(np_fraction_large_cluster_dividecluster[x,:])

    for y in range(0,np_fraction_large_cluster.shape[1]):
        np_fraction_large_cluster_divideregion[:,y] = np_fraction_large_cluster_divideregion[:,y]/np.sum(np_fraction_large_cluster_divideregion[:,y])





    fig,ax_list = plt.subplots(No_cluster_idx_large,1,figsize = (6,14),sharex =True,sharey =True,dpi=800)
    color_list = plt.cm.viridis(np.linspace(0,1,No_cluster_idx_large))
    for temp_id,i in zip(cluster_idx_large,range(0,No_cluster_idx_large)):
        ax_list[i].bar(region_list,np_fraction_large_cluster_dividecluster[i,:],color = color_list[i])
        ax_list[i].set_xticklabels(region_list, rotation = 40,fontsize = 14,horizontalalignment = 'right')
        ax_list[i].set_ylabel('Cluster'+str(int(temp_id)),rotation = 0,loc= 'bottom',fontsize = 14,labelpad = 70)
        ax_list[i].set_yticklabels([0,0.5],fontsize = 8)
    plt.suptitle(temp_fish+' '+temp_date+': Anatomical location of clusters',fontsize =15,y = 0.9)
    plt.show()


    fig,ax_list = plt.subplots(No_cluster_idx_large,1,figsize = (6,14),sharex =True,sharey =True,dpi=800)
    color_list = plt.cm.viridis(np.linspace(0,1,No_cluster_idx_large))
    for temp_id,i in zip(cluster_idx_large,range(0,No_cluster_idx_large)):
        ax_list[i].bar(region_list,np_fraction_large_cluster_divideregion[i,:],color = color_list[i])
        plt.ylim(0,0.3)
        ax_list[i].set_xticklabels(region_list, rotation = 40,fontsize = 14,horizontalalignment = 'right')
        ax_list[i].set_ylabel('Cluster'+str(int(temp_id)),rotation = 0,loc= 'bottom',fontsize = 14,labelpad = 70)
        ax_list[i].set_yticklabels([0,0.25],fontsize = 8)
    plt.suptitle(temp_fish+' '+temp_date+': Fraction of imaged cells within each anatomical region',fontsize =15,y = 0.9)   
    plt.show()